In [42]:

# SEAG qualification period is 22 Oct 2024 to 5 Sept 2025

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy



from google.cloud import storage



In [196]:
# BACKUP DATABASE
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT * FROM `saa-analytics.results.athlete_results_prod` 
"""

backup = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Backup DB/')


backup.to_csv('Backup_DB_June25.csv', sep=',', encoding='utf-8-sig', index=False)


Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1749]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [1750]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,23-Jun-96,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,None,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,23-Feb-03,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,None,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,26-Nov-94,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106


In [1751]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [1752]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,23-Jun-96,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,None,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,23-Feb-03,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,None,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,26-Nov-94,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,None,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106


In [1753]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
2189,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,1991,SGP,-,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International,None
2231,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,1991,SGP,-,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International,None
3826,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,1991,SGP,-,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International,None
22084,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,1991,SGP,-,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International,None


In [1754]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        if re.search('to|\s\-\s\d\s|\s\-\d\d', date):  # e.g. 03-04
              
            pos = re.search('to|\s\-\s\d', date)
            # Splice string to day and month

            split_pos_start=pos.start()+3
            


            final_date = date[split_pos_start:] # left string post splicing

            print(i, pos, date, final_date)
            final_year = year[2:]

            event_date = final_date + '/' + final_year

            print('old code', date, final_date, event_date)

            competitors.loc[rowIndex, 'event_date'] = event_date

        elif re.search('(\-\s\d\w)|(\-\s\d\d\w)', date):  # e.g. 18 - 19 January
                        
            pos = re.search('\-', date)  # from '-' onwards only
            # Splice string to day and month

            split_pos_start=pos.start()+2
            

            final_date = date[split_pos_start:] # left string post splicing

            
            final_year = year[2:]

            event_date = final_date + ' ' + final_year


            competitors.loc[rowIndex, 'event_date'] = event_date
            
        
    elif re.search('\w\-\w', date):
        
        if competitors.loc[rowIndex, 'COMPETITION'] == "National School Games":
            
            if competitors.loc[rowIndex, 'YEAR'] == '2024':
        
                event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd. 04 because event was in April 24 only
            
       #         print('NSG 2024', event_date)
        
                competitors.loc[rowIndex, 'event_date'] = event_date
            
            elif competitors.loc[rowIndex, 'YEAR'] == '2025':
                
                event_date = date + '-' +year[2:]
                
        #        print('NSG2025', event_date)
                
                competitors.loc[rowIndex, 'event_date'] = event_date
                
        elif re.search('\d\-\d',  date):        #10-13 April
            
            print('HERE', i, date)

            rpos = re.search('\-', date)
            string = date[rpos.end():]
            
            print('extracted date', string)
            
            event_date = string + ' ' + year
            
            print('event date', event_date)
            
            competitors.loc[rowIndex, 'event_date'] = event_date

            
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

0 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
1 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
2 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
3 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
4 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
5 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
6 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
7 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
8 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 June 8 June 8 June/25
11 <re.Match object; span=(1, 5), match=' - 8'> 7 - 8 June 8 June
old code 7 - 8 J

2273 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
2274 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
2275 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
2276 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
2277 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
2278 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
2279 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
2280 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
2281 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
2282 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/2

old code 3/13 to 3/17 3/17 3/17/24
4495 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
4496 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
4497 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
4498 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4499 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4500 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4501 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4502 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4503 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
4504 <re.Match object; span=(5, 7), match='to'> 3/13 to 

7027 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
7028 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
7029 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
7030 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
7031 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
7032 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
7033 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
7034 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
7035 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
7036 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/2

9929 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
9930 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
9931 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
9932 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
9933 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
9934 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
9935 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
9936 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
9937 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
9938 <re.Match object; span=(5, 7), match='to'> 4/18 to 4/19 4/19
old code 4/18 to 4/19 4/19 4/19/24
9939

...

17223 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
17224 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
17225 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
17226 <re.Match object; span=(5, 7), match='to'> 1/10 to 1/14 1/14
old code 1/10 to 1/14 1/14 1/14/24
17227 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17228 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17229 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17230 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17231 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 to 1/28 1/28 1/28/24
17232 <re.Match object; span=(5, 7), match='to'> 1/27 to 1/28 1/28
old code 1/27 t

19630 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
19631 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19632 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19633 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19634 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19635 <re.Match object; span=(5, 7), match='to'> 3/23 to 3/24 3/24
old code 3/23 to 3/24 3/24 3/24/24
19636 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19637 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19638 <re.Match object; span=(4, 6), match='to'> 3/9 to 3/9 3/9
old code 3/9 to 3/9 3/9 3/9/24
19639 <re.Match object; span=(5, 7), match='to'> 4/18 to 4/19 4/19
old code 4/18 to 4/19 4/19 4/19/24
1

21974 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21975 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21976 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21977 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21978 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21979 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21980 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21981 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
21982 <re.Match object; span=(5, 7), match='to'> 6/22 to 6/22 6/22
old code 6/22 to 6/22 6/22 6/22/24
21983 <re.Match object; span=(5, 7), match='to'> 6/22 to 6/22 6/22
old code 6/22 t

24423 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
24424 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
24425 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
24426 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24427 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24428 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24429 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24430 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24431 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 to 3/30 3/30 3/30/25
24432 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/30 3/30
old code 3/13 t

26577 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
26578 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
26579 <re.Match object; span=(4, 6), match='to'> 2/9 to 2/16 2/16
old code 2/9 to 2/16 2/16 2/16/25
26580 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26581 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26582 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26583 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26584 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26585 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17 3/17 3/17/24
26586 <re.Match object; span=(5, 7), match='to'> 3/13 to 3/17 3/17
old code 3/13 to 3/17

In [1755]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,SGP,None,Mid,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,SGP,None,Long,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,SGP,None,Jump,Male,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,SGP,None,Jump,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,SGP,None,Mid,Female,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,None,None,Relay,Male,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25


In [1756]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='mixed', dayfirst=False)

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month


In [1757]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0


In [1758]:
# Choose date range for SEAG qualification window from Oct 22 to current


#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

#competitors=competitors.reset_index(drop=True)

start = datetime.datetime(2024, 6, 23)
#start = datetime.datetime(2025, 5, 1)


end = datetime.datetime(2025, 6, 10)

start_date = np.datetime64(start)
end_date = np.datetime64(end)


mask = (competitors['event_date_dt'] >= start_date) & (competitors['event_date_dt'] <= end_date)
athletes_selected = competitors.loc[mask]



In [1759]:
athletes_selected.to_csv('athletes_downloaded_AY.csv', encoding='utf-8')

In [1760]:
# Select all of 2024/25 for OCTC

#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [1761]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0


In [1762]:
athletes_selected[athletes_selected['NAME']=='Hia, Caleb']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month


In [1763]:
# Choose 2024/25 only

athletes = athletes_selected

In [1764]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,Taiwan Athletics Open,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,National School Games,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0


In [1765]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month
18,Jun Jie Calvin Quek,50.94,None,None,8,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,31-May,2025,International,36:03.8,31-May-25,2025-05-31,18 days 12:36:47.074177,18.0,5.0
31,Jun Jie Calvin Quek,50.58,None,None,2,None,400m Hurdles,None,None,None,...,26th Asian Athletics Championships,27 - 31 May,2025,International,2025-06-01 16:49:06.835826,31 May 25,2025-05-31,18 days 12:36:47.074177,18.0,5.0
4333,Jun Jie Calvin Quek,51.64,None,None,6.0,None,400m Hurdles,None,None,None,...,12th Kinami Michitaka Memorial Athletics Meet,11-May,2025,International,2025-05-16 15:18:11.207945,11-May-25,2025-05-11,38 days 12:36:47.074177,38.0,5.0
5424,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,360 days 12:36:47.074177,360.0,6.0
7485,Jun Jie Calvin Quek,10.86,None,None,2,None,100m,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,63 days 12:36:47.074177,63.0,4.0
19361,Jun Jie Calvin Quek,50.77,None,None,1,None,400m Hurdles,None,None,None,...,Potch Invitational Meet,16-Apr,2025,International,None,16-Apr-25,2025-04-16,63 days 12:36:47.074177,63.0,4.0
22617,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Sydney Track Classic,15-Mar,2025,International,None,15-Mar-25,2025-03-15,95 days 12:36:47.074177,95.0,3.0
23631,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,None,23 June 24,2024-06-23,360 days 12:36:47.074177,360.0,6.0


In [1766]:
# Run events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Dash.+', value='200m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Dash.+', value='100m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Dash.+', value='400m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+800 Meter Run.+', value='800m')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Run.+', value='1500m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Run.+', value='3000m')

# Hurdles events

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110 Meter Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+110m Hurdles.+', value='110m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+100 Meter Hurdles.+', value='100m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+200 Meter Hurdles.+', value='200m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400 Meter Hurdles.+', value='400m hurdles')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+400m Hurdles.+', value='400m hurdles')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x100 Meter Relay.+', value='4 x 100m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+4x400 Meter Relay.+', value='4 x 400m relay')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 meter.+', value='3000m')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+1500 Meter Race Walk.+', value='1500m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000m Race Walk.+', value='3000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000m Race Walk.+', value='5000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Race Walk.+', value='10000m race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+5000 Meter Run.+', value='5000m run')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+10000 Meter Run.+', value='10000m run')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Race Walk.+', value='race walk')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Pole Vault.+', value='Pole vault')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Shot Put.+', value='Shot put')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Discus.+', value='Discus throw')

#mask = athletes['EVENT'].str.contains(r'Discus', na=True)
#athletes.loc[mask, 'EVENT'] = 'Discus throw'

#mask = athletes['EVENT'].str.contains(r'Shot', na=True)
#athletes.loc[mask, 'EVENT'] = 'Shot put'

#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'EVENT'] = 'Javelin throw'



#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Triple Jump.+', value='Triple jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Javelin Throw.+', value='Javelin throw')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+Long Jump.+', value='Long jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'Long Jump', value='Long jump')


#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'High Jump', value='High jump')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+High Jump.+', value='High jump')

#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+S/C.+', value='steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+3000 Meter Steeplechase.+', value='3000m steeplechase')
#athletes['EVENT'] = athletes['EVENT'].replace(regex=r'.+2000 Meter Steeplechase.+', value='2000m steeplechase')


#mask = athletes['EVENT'].str.contains(r'High', na=True)
#athletes.loc[mask, 'EVENT'] = 'High jump'

#mask = athletes['EVENT'].str.contains(r'110m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '110m hurdles'

#mask = athletes['EVENT'].str.contains(r'400m hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '400m hurdles'

#mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '200m hurdles'

#mask = athletes['EVENT'].str.contains(r'100m Hurdles', na=True)
#athletes.loc[mask, 'EVENT'] = '100m hurdles'

#mask = athletes['EVENT'].str.contains(r'4 X 100m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4 X 400m relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'

#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'Hammer Throw', na=True)
#athletes.loc[mask, 'EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '3000m steeplechase'

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'EVENT'] = '2000m steeplechase'


#mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 100m relay'

#mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
#athletes.loc[mask, 'EVENT'] = '4 x 400m relay'


# correct javelin category
#mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
#athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'




In [1767]:
# Wind mapping

#mask = athletes['EVENT'].str.contains(r'\d{1}.\d{1}', na=True, regex=True)
#athletes.loc[mask, 'WIND_MAP'] = '400m'


In [1768]:
# Create temporary mapped event column
# Junior Category Only

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
#         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
#         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914|91.4', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762|76.2cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84|0.838', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.762|0.84|0.838', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'500g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'700g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='3kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'5', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'3', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

#mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'5kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'3.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.5kg|1.50kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'U18', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m S\/C', na=True) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase|steeplechase|S\/C', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)  & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'2000m Steeplechase|2000m steeplechase|2000m S\/C', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'2000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3513542181.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [1769]:
athletes[(athletes['MAPPED_EVENT']=='Decathlon')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
5120,Jayden Ng,5716,None,None,3,None,Decathlon U18,None,None,None,...,22 - 23 March,2025,International,None,23 March 25,2025-03-23,87 days 12:36:47.074177,87.0,3.0,Decathlon


In [1770]:
athletes[athletes['MAPPED_EVENT']=='10,000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
29,Shaun Goh,32:21.0,None,None,16,None,"10,000m",None,None,None,...,27 - 31 May,2025,International,2025-06-01 16:49:06.835826,31 May 25,2025-05-31,18 days 12:36:47.074177,18.0,5.0,"10,000m"
588,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,20-Oct,2024,International,None,20-Oct-24,2024-10-20,241 days 12:36:47.074177,241.0,10.0,"10,000m"
1015,"Heng Chin Kiat, Richard",34:42.3,National University Singapore,0,2,Open,Run,10000,None,nan,...,1/8 to 1/12,2025,Local,None,1/12/25,2025-01-12,157 days 12:36:47.074177,157.0,1.0,"10,000m"
1016,"Chikani, Umaymah",44:53.2,National University Singapore,0,6,Open,Run,10000,None,nan,...,1/8 to 1/12,2025,Local,None,1/12/25,2025-01-12,157 days 12:36:47.074177,157.0,1.0,"10,000m"
1551,"Low, Nicole",37:37.7,Wings Athletics Club,27,4,Open,Run,10000,None,N328A98,...,2/9 to 2/16,2025,Local,None,2/16/25,2025-02-16,122 days 12:36:47.074177,122.0,2.0,"10,000m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25502,"Ranjan, Tharun",39:46.0,Nanyang Polytechnic,0,13,Open,Run,10000,None,nan,...,1/8 to 1/12,2025,Local,None,1/12/25,2025-01-12,157 days 12:36:47.074177,157.0,1.0,"10,000m"
25528,"Mani, Manoj",40:19.8,Singapore Institute of Managem,25,9,Open,Run,10000,None,M323B00,...,2/9 to 2/16,2025,Local,None,2/16/25,2025-02-16,122 days 12:36:47.074177,122.0,2.0,"10,000m"
25554,"Tan, Bernice",44:25.6,Lacticbuds,25,3,Open,Run,10000,None,B075C00,...,4/24 to 4/25,2025,Local,None,4/25/25,2025-04-25,54 days 12:36:47.074177,54.0,4.0,"10,000m"
26553,"Ganesan, Kathiravan",40:39.6,Singapore Institute of Technol,0,14,Open,Run,10000,None,nan,...,1/8 to 1/12,2025,Local,None,1/12/25,2025-01-12,157 days 12:36:47.074177,157.0,1.0,"10,000m"


In [1771]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/2203170124.py:5: SettingWithCopyWarning: 


In [1772]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


athletes.to_csv('athletes_post_map_AY.csv', sep=',', encoding='utf-8-sig', index=False)


In [1773]:
# Load Asian Youth Benchmarks

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Benchmarks/')

all_benchmark = pd.read_csv("All_Benchmarks.csv")




In [1774]:
all_benchmark

,BENCHMARK,EVENT,GENDER,RESULT
0,26th Asian Athletics,100m,Male,10.33
1,26th Asian Athletics,100m,Female,11.66
2,26th Asian Athletics,200m,Male,21.01
3,26th Asian Athletics,200m,Female,23.66
4,26th Asian Athletics,400m,Male,46.29
...,...,...,...,...
159,2025 Asian Youth,5000m Race Walk,Female,28:58.25
160,2025 Asian Youth,Decathlon,Male,5249.00
161,2025 Asian Youth,Heptathlon,Female,3965.00
162,2025 Asian Youth,Medley Relay,Male,01:59.16


In [1775]:
benchmark=all_benchmark[all_benchmark['BENCHMARK']=='2025 Asian Youth']

In [1776]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT
124,2025 Asian Youth,100m,Male,10.85
125,2025 Asian Youth,100m,Female,12.25
126,2025 Asian Youth,200m,Male,22.25
127,2025 Asian Youth,200m,Female,25.00
128,2025 Asian Youth,400m,Male,49.75
129,2025 Asian Youth,400m,Female,59.12
130,2025 Asian Youth,800m,Male,02:00.12
131,2025 Asian Youth,800m,Female,02:23.21
132,2025 Asian Youth,1500m,Male,04:08.48
133,2025 Asian Youth,1500m,Female,04:53.05


In [1777]:
benchmark=benchmark.reset_index(drop=True)

In [1778]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                elif '5000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                    
                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [1779]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,3]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [1780]:
process_benchmarks(benchmark)

0 2025 Asian Youth 10.85
1 2025 Asian Youth 12.25
2 2025 Asian Youth 22.25
3 2025 Asian Youth 25.0
4 2025 Asian Youth 49.75
5 2025 Asian Youth 59.12
6 2025 Asian Youth 120.12
7 2025 Asian Youth 143.21
8 2025 Asian Youth 248.48
9 2025 Asian Youth 293.05
10 2025 Asian Youth 551.87
11 2025 Asian Youth 656.29
12 2025 Asian Youth 386.83
13 2025 Asian Youth 468.16
14 2025 Asian Youth 14.25
15 2025 Asian Youth 15.53
16 2025 Asian Youth 56.39
17 2025 Asian Youth 67.36
18 2025 Asian Youth 1.97
19 2025 Asian Youth 1.62
20 2025 Asian Youth 6.94
21 2025 Asian Youth 5.57
22 2025 Asian Youth 13.63
23 2025 Asian Youth 11.88
24 2025 Asian Youth 16.69
25 2025 Asian Youth 12.74
26 2025 Asian Youth 49.94
27 2025 Asian Youth 33.99
28 2025 Asian Youth 52.57
29 2025 Asian Youth 33.6
30 2025 Asian Youth 58.78
31 2025 Asian Youth 40.61
32 2025 Asian Youth 4.03
33 2025 Asian Youth 3.08
34 2025 Asian Youth 2875.22
35 2025 Asian Youth 1738.25
36 2025 Asian Youth 5249.0
37 2025 Asian Youth 3965.0
38 2025 Asian Yo

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [1781]:
'''
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,3]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out
'''

"\nfor i in range(len(benchmarks)):\n        \n    rowIndex = benchmarks.index[i]\n\n    input_string=benchmarks.iloc[rowIndex,0]\n    \n    metric=benchmarks.iloc[rowIndex,3]\n    \n    if metric==None:\n        continue\n        \n    out = convert_time(i, input_string, metric)\n    \n    print(rowIndex, input_string, out)\n     \n    benchmarks.loc[rowIndex, 'Metric'] = out\n"

In [1782]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric
0,2025 Asian Youth,100m,Male,10.85,10.85
1,2025 Asian Youth,100m,Female,12.25,12.25
2,2025 Asian Youth,200m,Male,22.25,22.25
3,2025 Asian Youth,200m,Female,25.00,25.00
4,2025 Asian Youth,400m,Male,49.75,49.75
5,2025 Asian Youth,400m,Female,59.12,59.12
6,2025 Asian Youth,800m,Male,02:00.12,120.12
7,2025 Asian Youth,800m,Female,02:23.21,143.21
8,2025 Asian Youth,1500m,Male,04:08.48,248.48
9,2025 Asian Youth,1500m,Female,04:53.05,293.05


In [1783]:
mask = benchmark['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmark.loc[mask, '2%']=benchmark['Metric']*0.98
benchmark.loc[mask, '3.5%']=benchmark['Metric']*0.965
benchmark.loc[mask, '5%']=benchmark['Metric']*0.95

benchmark.loc[~mask, '2%']=benchmark['Metric']*1.02
benchmark.loc[~mask, '3.5%']=benchmark['Metric']*1.035
benchmark.loc[~mask, '5%']=benchmark['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [1784]:
benchmark['MAPPED_EVENT']=benchmark['EVENT'].str.strip()

In [1785]:
for col in benchmark.columns:
    benchmark[col] = benchmark[col].astype(str)
    benchmark[col] = benchmark[col].str.replace('\xa0', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmark[col] = benchmark[col].str.replace('\r', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('\n', ' ', regex=True)
    benchmark[col] = benchmark[col].str.strip()


In [1786]:
benchmark

,BENCHMARK,EVENT,GENDER,RESULT,Metric,2%,3.5%,5%,MAPPED_EVENT
0,2025 Asian Youth,100m,Male,10.85,10.85,11.067,11.22975,11.3925,100m
1,2025 Asian Youth,100m,Female,12.25,12.25,12.495000000000001,12.678749999999999,12.8625,100m
2,2025 Asian Youth,200m,Male,22.25,22.25,22.695,23.02875,23.3625,200m
3,2025 Asian Youth,200m,Female,25.00,25.0,25.5,25.874999999999996,26.25,200m
4,2025 Asian Youth,400m,Male,49.75,49.75,50.745,51.491249999999994,52.237500000000004,400m
5,2025 Asian Youth,400m,Female,59.12,59.12,60.3024,61.18919999999999,62.076,400m
6,2025 Asian Youth,800m,Male,02:00.12,120.12,122.5224,124.32419999999999,126.126,800m
7,2025 Asian Youth,800m,Female,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,800m
8,2025 Asian Youth,1500m,Male,04:08.48,248.48,253.4496,257.17679999999996,260.904,1500m
9,2025 Asian Youth,1500m,Female,04:53.05,293.05,298.911,303.30674999999997,307.70250000000004,1500m


In [1787]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0,800m
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0,5000m
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0,Triple Jump
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0,Pole Vault
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,7 - 8 June,2025,International,2025-06-11 16:38:50.923703,8 June/25,2025-06-08,10 days 12:36:47.074177,10.0,6.0,800m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31154,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0,4 x 400m
31155,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0,4 x 400m
31156,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0,4 x 400m
31157,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,28-04,2025,Local,2025-06-06 15:35:30.807106,28-04-25,2025-04-28,51 days 12:36:47.074177,51.0,4.0,4 x 400m


In [1788]:
athletes[athletes['NAME']=='Heng, Richard']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,TIMESTAMP,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT


In [1789]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [1790]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmark,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [1791]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,10.0,6.0,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,10.0,6.0,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,10.0,6.0,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,10.0,6.0,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,10.0,6.0,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1792]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%


In [1793]:
df[df['MAPPED_EVENT']=='Decathlon']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
2440,Jayden Ng,5716,None,None,3,None,Decathlon U18,None,None,None,...,87.0,3.0,Decathlon,2025 Asian Youth,Decathlon,5249.00,5249.0,5144.0199999999995,5065.285,4986.55


In [1794]:
# replace '-' with NaN

df['RESULT_x'] = df['RESULT_x'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [1795]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,10.0,6.0,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,10.0,6.0,5000m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,10.0,6.0,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,10.0,6.0,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,10.0,6.0,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,51.0,4.0,4 x 400m,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1796]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%


In [1797]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('Asian_Youth_postmap.csv', sep=',', encoding='utf-8-sig', index=False)


In [1798]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)

for col in df.columns:
    
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()

for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    event=df.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=df.loc[rowIndex,'RESULT_x'] # result
    
    if result=='—' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/201318515.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out


In [1799]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [1800]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,6.0,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,6.0,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,6.0,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,6.0,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,6.0,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,4.0,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,4.0,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,4.0,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,4.0,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4


In [1801]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [1802]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:00.12,120.12,122.5224,124.32419999999999,126.126,114.0,114.0
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,5000m,nan,nan,nan,nan,nan,nan,nan,1114.34,1114.34
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,Triple Jump,2025 Asian Youth,Triple Jump,13.63,13.63,13.3574,13.15295,12.948500000000001,15.15,15.15
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,Pole Vault,2025 Asian Youth,Pole Vault,3.08,3.08,3.0184,2.9722,2.9259999999999997,3.2,3.2
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,800m,2025 Asian Youth,800m,02:23.21,143.21,146.07420000000002,148.22235,150.37050000000002,136.9,136.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.06,223.06
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,223.67,223.67
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,225.03,225.03
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,4 x 400m,nan,nan,nan,nan,nan,nan,nan,227.4,227.4


In [1803]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,BENCHMARK,EVENT_y,RESULT_y,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST


In [1804]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric, errors='coerce')

In [1805]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|heptathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [1806]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [1807]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv('AY_postmap_benchmarked.csv', sep=',', encoding='utf-8-sig', index=False)


In [1808]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
25,Shaun Goh,32:21.0,None,None,16,None,"10,000m",None,None,None,...,NaN,NaN,NaN,1941.0,1941.0,NaN,NaN,NaN,NaN,NaN
311,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,NaN,NaN,NaN,2274.0,2274.0,NaN,NaN,NaN,NaN,NaN
509,"Heng Chin Kiat, Richard",34:42.3,National University Singapore,0,2,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2082.3,2082.3,NaN,NaN,NaN,NaN,NaN
510,"Chikani, Umaymah",44:53.2,National University Singapore,0,6,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2693.2,2693.2,NaN,NaN,NaN,NaN,NaN
765,"Low, Nicole",37:37.7,Wings Athletics Club,27,4,Open,Run,10000,None,N328A98,...,NaN,NaN,NaN,2257.7,2257.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12255,"Ranjan, Tharun",39:46.0,Nanyang Polytechnic,0,13,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2386.0,2386.0,NaN,NaN,NaN,NaN,NaN
12271,"Mani, Manoj",40:19.8,Singapore Institute of Managem,25,9,Open,Run,10000,None,M323B00,...,NaN,NaN,NaN,2419.8,2419.8,NaN,NaN,NaN,NaN,NaN
12288,"Tan, Bernice",44:25.6,Lacticbuds,25,3,Open,Run,10000,None,B075C00,...,NaN,NaN,NaN,2665.6,2665.6,NaN,NaN,NaN,NaN,NaN
12780,"Ganesan, Kathiravan",40:39.6,Singapore Institute of Technol,0,14,Open,Run,10000,None,nan,...,NaN,NaN,NaN,2439.6,2439.6,NaN,NaN,NaN,NaN,NaN


In [1809]:
df

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,Nicole Low Sui Xuan,18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Gabriel Lee,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,Esther Tay Shee Wei,3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew's Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [1810]:
'''
# Read a variation name list and corrections from CSVs

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/')

names = pd.read_csv("junior_names.csv")

for index, row in names.iterrows():
        
    print(names.VARIATION, names.NAME)
    df['NAME_MAPPED'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
'''

'\n# Read a variation name list and corrections from CSVs\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/Name Variations/\')\n\nnames = pd.read_csv("junior_names.csv")\n\nfor index, row in names.iterrows():\n        \n    print(names.VARIATION, names.NAME)\n    df[\'NAME_MAPPED\'] = df[\'NAME\'].replace(regex=rf"{row[\'VARIATION\']}", value=f"{row[\'NAME\']}")\n'

In [1811]:
# Read name variations from GCS name lists bucket (Still in beta)

df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()

df['NAME'] = df['NAME'].str.casefold()  # everything lower case


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


# Clean and apply casefold to name variations

names['NAME'] = names['NAME'].str.replace('\xa0', '', regex=True)
names['NAME'] = names['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\r', '', regex=True)
names['NAME'] = names['NAME'].str.replace('\n', '', regex=True)
names['NAME'] = names['NAME'].str.strip()
names['VARIATION'] = names['VARIATION'].str.replace('\xa0', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\r', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.replace('\n', '', regex=True)
names['VARIATION'] = names['VARIATION'].str.strip()

names['VARIATION'] = names['VARIATION'].str.casefold() # convert to lower case
names['NAME'] = names['NAME'].str.casefold()


# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")
    

    
df['NAME'] = df['NAME'].str.title()  # capitalize first letter


In [1812]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


df.to_csv("names_check.csv", encoding='utf-8')

In [1813]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China')&(df['TEAM']!='Thailand') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos')&(df['TEAM']!='Myanmar') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')&(df['TEAM']!='Vietnam')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [1814]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [1815]:
df_select[df_select['NAME']=='FUN LE CONG LUCAS']

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR


In [1816]:
df_select[(df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m')]

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
12620,"Tan, Kian Ye",26.19,Erovra Club,12,1,U13,Dash,200,None,K546A13,...,22.695,23.02875,23.3625,26.19,26.19,-3.495,-3.16125,-2.8275,-3.94,-12.707865


In [1817]:
# Remove spurious SEED results 

#df_select=df_select[~((df_select['NAME']=='Tan, Kian Ye') & (df_select['MAPPED_EVENT']=='200m'))]
#df_select=df_select[~((df_select['NAME']=='LOH, BEAVERLY') & (df_select['MAPPED_EVENT']=='400m'))]
#df_select=df_select[~((df_select['NAME']=='Vaz, Luisa Marie') & (df_select['MAPPED_EVENT']=='High jump'))]


In [1818]:
df_select

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [1819]:
'''
# Read list of foreigners

foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')
'''


"\n# Read list of foreigners\n\nforeigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')\n"

In [1820]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [1821]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
235,Kashama,Biwesa Daniel
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian


In [1822]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [1823]:
#rslt_df = df.loc[(df['RESULT_CONV'] < df['2pc']) & (df['AGE']<40) & (df['EVENT']!='Marathon')]

In [1824]:
#rslt_df = df.loc[(((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))&(df['RESULT_CONV'] >= df['5pc']) & (df['AGE']<40) & ((df['EVENT']!='Marathon')|(df['AGE']<60) & (df['EVENT']=='Marathon')))]

In [1825]:
no_foreigners_list

,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,Zubin Percy Muncherji,01:54.0,None,None,2.0,None,800m,None,None,None,...,122.5224,124.32420,126.1260,114.0,114.00,8.5224,10.32420,12.1260,6.12,10.094905
1,"Low, Nicole Sui Xuan",18:34.34,None,None,6.0,None,5000m,None,None,None,...,NaN,NaN,NaN,1114.34,1114.34,NaN,NaN,NaN,NaN,NaN
2,Lee Gabriel Jin Yi,15.15,None,None,7.0,None,Triple Jump,None,None,None,...,13.3574,13.15295,12.9485,15.15,15.15,1.7926,1.99705,2.2015,1.52,16.151871
3,"Tay, Esther",3.2,None,None,10.0,None,Pole Vault,None,None,None,...,3.0184,2.97220,2.9260,3.2,3.20,0.1816,0.22780,0.2740,0.12,8.896104
4,Romaine Soh Rui Min,02:16.90,None,None,5.0,None,800m,None,None,None,...,146.0742,148.22235,150.3705,136.9,136.90,9.1742,11.32235,13.4705,6.31,9.406117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,St. Andrew'S Junior College,03:43.06,SAJC,None,5.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.06,223.06,NaN,NaN,NaN,NaN,NaN
17329,Catholic Junior College,03:43.67,CJC,None,6.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,223.67,223.67,NaN,NaN,NaN,NaN,NaN
17330,Yishun Innova Junior College,03:45.03,YIJC,None,7.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,225.03,225.03,NaN,NaN,NaN,NaN,NaN
17331,National Junior College,03:47.40,NJC,None,8.0,A,4x400m Relay,None,None,None,...,NaN,NaN,NaN,227.4,227.40,NaN,NaN,NaN,NaN,NaN


In [1826]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [1827]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [1828]:
top_performers_clean.reset_index(inplace=True)


In [1829]:
# Process dates to extract age

# Map NSG divisions into age

mask = (top_performers_clean['DIVISION'].str.contains(r'A', na=False))
top_performers_clean.loc[mask, 'AGE'] = '18.5'

mask = (top_performers_clean['DIVISION'].str.contains(r'B', na=False))
top_performers_clean.loc[mask, 'AGE'] = '16'

mask = (top_performers_clean['DIVISION'].str.contains(r'C', na=False))
top_performers_clean.loc[mask, 'AGE'] = '13.5'

mask = (top_performers_clean['DIVISION'].str.contains(r'O', na=False))
top_performers_clean.loc[mask, 'AGE'] = '12'


In [1830]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


top_performers_clean['DOB_new'] = top_performers_clean['DOB'].apply(length)




In [1831]:
top_performers_clean['DOB_new'] = pd.to_datetime(top_performers_clean['DOB_new'], errors='coerce')

top_performers_clean['year_extract'] = top_performers_clean['DOB_new'].dt.strftime('%Y')

top_performers_clean['year_extract'] = pd.to_numeric(top_performers_clean['year_extract'])

top_performers_clean['age_extract'] = 2025 - top_performers_clean['year_extract']


In [1832]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


top_performers_clean['age_extract']=top_performers_clean['age_extract'].apply(age)


In [1833]:
condition1 = top_performers_clean['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


top_performers_clean['age_extract'] = top_performers_clean['AGE'].where((condition1), top_performers_clean['age_extract'].values)


In [1834]:
top_performers_clean[['age_extract']] = top_performers_clean[['age_extract']].apply(pd.to_numeric)

In [1835]:
top_performers_clean

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,NaT,NaN,13.5
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,2005-10-18,2005.0,20.0
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,2011-02-21,2011.0,14.0
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,NaT,NaN,13.5
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,2011-07-25,2011.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-21,2016.0,9.0
11409,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-04,2018.0,7.0
11410,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-04,2010.0,15.0
11411,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2010.0,15.0


In [1836]:
# Filter out names with no DOB

dob_df = top_performers_clean[top_performers_clean['DOB_new'].notnull()]

In [1837]:
dob_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,2005-10-18,2005.0,20.0
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,2011-02-21,2011.0,14.0
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,2011-07-25,2011.0,14.0
6,9997,"Zhang, Xinyu",7.95m,Chij St. Nicholas Girls',13,15,U15,Triple Jump,0,None,...,7.95,7.95,-3.6924,-3.51420,-3.3360,-3.93,-28.080808,2012-02-24,2012.0,13.0
7,5343,"Zhang, Jiuyuan",10.75m,Catholic High School,15,25,U18,Triple Jump,0,None,...,10.75,10.75,-2.6074,-2.40295,-2.1985,-2.88,-16.129861,2010-12-09,2010.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11407,5524,"., Shawn",05:29.5,North Vista,12,33,Open,Mile Run,1,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2010.0,15.0
11408,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-21,2016.0,9.0
11409,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-04,2018.0,7.0
11410,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-04,2010.0,15.0


In [1875]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')

dob_df.to_csv('dob_df.csv', encoding='utf-8')

In [1838]:
# Create dictionary of names, dobs for everyone

dictionary_dob = dict(zip(dob_df['NAME'].str.casefold(), dob_df['DOB_new']))

In [1874]:
dictionary_dob

{'zhong, chuhan': Timestamp('2005-10-18 00:00:00'),
 'zheng, justin de': Timestamp('2011-02-21 00:00:00'),
 'zhao, daniel': Timestamp('2008-07-25 00:00:00'),
 'zhang, xinyu': Timestamp('2012-02-24 00:00:00'),
 'zhang, jiuyuan': Timestamp('2010-12-09 00:00:00'),
 'yuki, ho you shu': Timestamp('2006-08-30 00:00:00'),
 'yuen, tess shu ling': Timestamp('2009-02-12 00:00:00'),
 'yuan, yinze': Timestamp('2008-09-06 00:00:00'),
 'yong, zeheng': Timestamp('2010-04-08 00:00:00'),
 'yew, meredith': Timestamp('2006-09-28 00:00:00'),
 'yeo, natasha': Timestamp('2007-01-31 00:00:00'),
 'yeo, husni': Timestamp('2005-02-01 00:00:00'),
 'yen, ming zhen': Timestamp('2008-01-24 00:00:00'),
 'yap, shauna sze en': Timestamp('2007-11-14 00:00:00'),
 'yap, leia': Timestamp('2007-08-27 00:00:00'),
 'yang, shihao': Timestamp('2010-03-30 00:00:00'),
 'yang, en hao daniel': Timestamp('2007-01-15 00:00:00'),
 'xu, weixin': Timestamp('1985-06-01 00:00:00'),
 'xiyuan, li': Timestamp('2009-03-13 00:00:00'),
 'wu, x

In [1842]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


top_performers_clean.to_csv('AY_top_performers_prod.csv', encoding='utf-8')

In [1843]:
top_performers_clean

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,NaT,NaN,13.5
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,2005-10-18,2005.0,20.0
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,2011-02-21,2011.0,14.0
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,NaT,NaN,13.5
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,2011-07-25,2011.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-21,2016.0,9.0
11409,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-04,2018.0,7.0
11410,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-04,2010.0,15.0
11411,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2010.0,15.0


In [1844]:
'''
# Join 2024 best results for each event for each athlete

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/')


df_yoy = pd.read_csv("Best_results_2023.csv")

df_yoy['NAME'] = df_yoy['NAME'].str.replace('\xa0', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\r', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.replace('\n', '', regex=True)
df_yoy['NAME'] = df_yoy['NAME'].str.strip()
'''

'\n# Join 2024 best results for each event for each athlete\n\nos.chdir(\'/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/2023/\')\n\n\ndf_yoy = pd.read_csv("Best_results_2023.csv")\n\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\xa0\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'[\x00-\x1f\x7f-\x9f]\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\r\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.replace(\'\n\', \'\', regex=True)\ndf_yoy[\'NAME\'] = df_yoy[\'NAME\'].str.strip()\n'

In [1845]:
#df_yoy

In [1846]:
'''

# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

yoy_performance = pd.merge(
    left=df_yoy, 
    right=top_performers_clean,
    how='left',
    left_on=['EVENT', 'GENDER', 'NAME'],
    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],
)

'''

"\n\n# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER\n\nyoy_performance = pd.merge(\n    left=df_yoy, \n    right=top_performers_clean,\n    how='left',\n    left_on=['EVENT', 'GENDER', 'NAME'],\n    right_on=['MAPPED_EVENT', 'GENDER', 'NAME'],\n)\n\n"

In [1847]:
#yoy_performance

In [1848]:
'''

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


yoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')

'''

"\n\nos.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')\n\n\nyoy_performance.to_csv('yoy_performance_prod.csv', index=False, encoding='utf-8')\n\n"

In [1849]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [1850]:
tiered_performers

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,9.12,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,NaT,NaN,13.5
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,12.26,0.6176,0.79580,0.9740,0.38,8.198653,2005-10-18,2005.0,20.0
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,10.47,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,2011-02-21,2011.0,14.0
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,10.29,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,NaT,NaN,13.5
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,11.67,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,2011-07-25,2011.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-21,2016.0,9.0
11409,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-04,2018.0,7.0
11410,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-04,2010.0,15.0
11411,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2010.0,15.0


In [1851]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [1852]:
tiered_performers[tiered_performers['MAPPED_EVENT']=='Marathon']

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract,TIER
574,3518,"Yong, Hui Lin",04:25:29,None,None,8098,None,Marathon,None,None,...,15929.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,
575,11461,Yik Siong Wong,2:46:37,INDIVIDUAL,29,None,None,Marathon,None,None,...,9997.0,NaN,NaN,NaN,NaN,NaN,1995-06-18,1995.0,30.0,
576,5253,"Ye, Pu",03:28:22,None,None,1658,None,Marathon,None,None,...,12502.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,
577,212,Yaohan Melvin Wong,2:34:33,None,None,42,None,Marathon,None,None,...,9273.0,NaN,NaN,NaN,NaN,NaN,1983-08-27,1983.0,42.0,
578,7318,Xiuying Hu,3:10:29,INDIVIDUAL,37,None,None,Marathon,None,None,...,11429.0,NaN,NaN,NaN,NaN,NaN,1987-06-18,1987.0,38.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,5766,"Chan, Jeremy Wei Lun",2:40:21,INDIVIDUAL,35,1064,None,Marathon,None,None,...,9621.0,NaN,NaN,NaN,NaN,NaN,1989-06-18,1989.0,36.0,
654,1518,"Chan, Choon Meng",05:23:50,None,None,26307,None,Marathon,None,None,...,19430.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,
655,12235,Ansgar Cheng,2:51:45,INDIVIDUAL,58,None,None,Marathon,None,None,...,10305.0,NaN,NaN,NaN,NaN,NaN,1966-06-18,1966.0,59.0,
656,1754,"Ang, Kok Kiong",03:51:54,None,None,10853,None,Marathon,None,None,...,13914.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,


In [1853]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [1854]:
'''
# Process dates to extract age

# Map NSG divisions into age

mask = (final_df['DIVISION'].str.contains(r'A', na=False))
final_df.loc[mask, 'AGE'] = '18.5'

mask = (final_df['DIVISION'].str.contains(r'B', na=False))
final_df.loc[mask, 'AGE'] = '16'

mask = (final_df['DIVISION'].str.contains(r'C', na=False))
final_df.loc[mask, 'AGE'] = '13.5'

mask = (final_df['DIVISION'].str.contains(r'O', na=False))
final_df.loc[mask, 'AGE'] = '12'

'''

"\n# Process dates to extract age\n\n# Map NSG divisions into age\n\nmask = (final_df['DIVISION'].str.contains(r'A', na=False))\nfinal_df.loc[mask, 'AGE'] = '18.5'\n\nmask = (final_df['DIVISION'].str.contains(r'B', na=False))\nfinal_df.loc[mask, 'AGE'] = '16'\n\nmask = (final_df['DIVISION'].str.contains(r'C', na=False))\nfinal_df.loc[mask, 'AGE'] = '13.5'\n\nmask = (final_df['DIVISION'].str.contains(r'O', na=False))\nfinal_df.loc[mask, 'AGE'] = '12'\n\n"

In [1855]:
'''
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final_df['DOB'].apply(length)



#B = parser.parse("10-09-2021", dayfirst = True)

'''

'\ndef length(string):\n\n    B = \'\'\n    year = \'\'\n\n    try:\n\n        length = len(string)\n\n        if length == 2:\n\n            string = \'19\' + string\n\n        elif length == 1:\n\n            string = \'\'\n\n        else:\n\n            pass\n\n        if string is not None or len(string) != 1:\n\n            B = parser.parse(string, dayfirst=True)\n                        \n    except:\n\n        pass\n\n    return B\n\n\nfinal_df[\'DOB_new\'] = final_df[\'DOB\'].apply(length)\n\n\n\n#B = parser.parse("10-09-2021", dayfirst = True)\n\n'

In [1856]:
'''
final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')

final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']
'''

"\nfinal_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')\n\nfinal_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')\n\nfinal_df['year_extract'] = pd.to_numeric(final_df['year_extract'])\n\nfinal_df['age_extract'] = 2025 - final_df['year_extract']\n"

In [1858]:
'''
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)
'''

"\ndef age(number):  # correct negative age numbers\n    \n    if number<0:\n        \n        number+=100\n        \n    return number\n\n\nfinal_df['age_extract']=final_df['age_extract'].apply(age)\n"

In [1859]:
'''

# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)
'''

"\n\n# If NSG event then choose AGE otherwise choose age_extract\n\ncondition1 = final_df['COMPETITION']=='National School Games'\n#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))\n#condition3=df['SEED_CONV']<df['RESULT_CONV']\n#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))\n\n\nfinal_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)\n"

In [1860]:
'''

# Change to numeric

final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

'''

"\n\n# Change to numeric\n\nfinal_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)\n\n"

In [1861]:
final_df

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract,TIER
0,16545,Zhou Xuanyu,9.12,DHS,13.5,10.0,C,Triple Jump,None,None,...,9.12,-2.5224,-2.34420,-2.1660,-2.76,-18.232323,NaT,NaN,13.5,
1,11022,"Zhong, Chuhan",12.26,None,None,None,None,Triple Jump,None,None,...,12.26,0.6176,0.79580,0.9740,0.38,8.198653,2005-10-18,2005.0,20.0,Tier 1
2,1076,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,10.47,-2.8874,-2.68295,-2.4785,-3.16,-18.184153,2011-02-21,2011.0,14.0,
3,12853,Zheng Justin De,10.29,NJC,13.5,12.0,C,Triple Jump,None,None,...,10.29,-3.0674,-2.86295,-2.6585,-3.34,-19.504769,NaT,NaN,13.5,
4,2326,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,11.67,-1.6874,-1.48295,-1.2785,-1.96,-9.380044,2011-07-25,2011.0,14.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408,3879,"., Shaik Isa",15.49,Club ZOOM,8,32,U9,Dash,80,,...,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-21,2016.0,9.0,
11409,7899,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-04,2018.0,7.0,
11410,9907,"., Dharkshitha",08:35.6,Cedar Girls Secondary School,15,1,U18,Race Walk,1500,None,...,NaN,NaN,NaN,NaN,NaN,NaN,2010-06-04,2010.0,15.0,
11411,3520,"., Cayden",06:05.1,North Vista,12,54,Open,Mile Run,1,,...,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2010.0,15.0,


In [1862]:
# Choose rows with mapped event

final_df=final_df[(final_df['MAPPED_EVENT']!=' ') & (final_df['TIER']!=' ')]

In [1863]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_df.to_csv('AY_tiered_performers.csv', encoding='utf-8')

In [1864]:
mask = (((final_df['age_extract'] >= 14) & (final_df['age_extract'] <= 16))|(final_df['age_extract'].isnull()))
#mask = ((final_df['age_extract'].isnull()))


final_selection = final_df.loc[mask]


In [1865]:
final_selection

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract,TIER
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,NaT,NaN,NaN,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,14.17,0.8126,1.01705,1.2215,0.54,8.961849,2009-07-15,2009.0,16.0,Tier 1
100,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,13.92,0.5626,0.76705,0.9715,0.29,7.127660,NaT,NaN,NaN,Tier 1
151,4356,"Medina, Andrew George",15.13,None,None,5,None,Triple Jump,None,None,...,15.13,1.7726,1.97705,2.1815,1.50,16.005136,NaT,NaN,NaN,Tier 1
365,12819,"Loh Ding Rong, Anson Ding Rong",17.2,None,None,1,None,Shot Put,None,5kg,...,17.20,0.8438,1.09415,1.3445,0.51,8.055722,NaT,NaN,NaN,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8167,2847,"Huang, Weijun",11.1,None,None,4,None,100m,None,None,...,11.10,-0.0330,0.12975,0.2925,-0.25,2.695853,NaT,NaN,NaN,Tier 3
8184,8441,"Ho, Xander",11.02,Nanyang Technological Universi,12,3,Open,Dash,100,None,...,11.02,0.0470,0.20975,0.3725,-0.17,3.433180,NaT,NaN,NaN,Tier 2
8213,3343,Hakim Muhammad Aryan,11.23,None,None,1,None,100m,None,None,...,11.23,-0.1630,-0.00025,0.1625,-0.38,1.497696,2010-06-18,2010.0,15.0,Tier 4
8258,2126,"Fang, Kayden",11.37,VICTORIA SCHOOL,16,4,U18,Dash,100,None,...,11.37,-0.3030,-0.14025,0.0225,-0.52,0.207373,2009-02-19,2009.0,16.0,Tier 4


In [1866]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Asian Youth/')


final_selection.to_csv('AY_final_selection.csv', encoding='utf-8')

In [1867]:

final_selection

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,DOB_new,year_extract,age_extract,TIER
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,11.59,-0.0524,0.12580,0.3040,-0.29,2.558923,NaT,NaN,NaN,Tier 3
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,14.17,0.8126,1.01705,1.2215,0.54,8.961849,2009-07-15,2009.0,16.0,Tier 1
100,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,13.92,0.5626,0.76705,0.9715,0.29,7.127660,NaT,NaN,NaN,Tier 1
151,4356,"Medina, Andrew George",15.13,None,None,5,None,Triple Jump,None,None,...,15.13,1.7726,1.97705,2.1815,1.50,16.005136,NaT,NaN,NaN,Tier 1
365,12819,"Loh Ding Rong, Anson Ding Rong",17.2,None,None,1,None,Shot Put,None,5kg,...,17.20,0.8438,1.09415,1.3445,0.51,8.055722,NaT,NaN,NaN,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8167,2847,"Huang, Weijun",11.1,None,None,4,None,100m,None,None,...,11.10,-0.0330,0.12975,0.2925,-0.25,2.695853,NaT,NaN,NaN,Tier 3
8184,8441,"Ho, Xander",11.02,Nanyang Technological Universi,12,3,Open,Dash,100,None,...,11.02,0.0470,0.20975,0.3725,-0.17,3.433180,NaT,NaN,NaN,Tier 2
8213,3343,Hakim Muhammad Aryan,11.23,None,None,1,None,100m,None,None,...,11.23,-0.1630,-0.00025,0.1625,-0.38,1.497696,2010-06-18,2010.0,15.0,Tier 4
8258,2126,"Fang, Kayden",11.37,VICTORIA SCHOOL,16,4,U18,Dash,100,None,...,11.37,-0.3030,-0.14025,0.0225,-0.52,0.207373,2009-02-19,2009.0,16.0,Tier 4


In [1879]:
# Get list of names with no DOB

mask = ((final_selection['DOB_new'].isnull()))

final_names = final_selection.loc[mask]

name_list = final_names['NAME'].str.casefold().to_list()


In [1880]:
name_list

['tse teng tan',
 'saw, xiang yu',
 'medina, andrew george',
 'loh ding rong, anson ding rong',
 'foo ming zhe, sean',
 'toh jun xi, tedd',
 'han lin feng',
 'chua, garrett',
 'lam xin, kristel',
 'jin rou wong',
 'lee yu foong (li yufeng)',
 'wee, nicholas',
 'tay, kai bin',
 'saicharan ganeshkumar',
 'lim ying qi, eiffer',
 'lim tze rong, oliver',
 'kuah yong chuan, ezekiel',
 'jeyaseelan sai hayagreev',
 'deianira jeffrey',
 'veroy chua jia hao',
 'tng, kai xin',
 'mayah elizabeth kaufmann',
 'yeo shin kee (yang xuanqi)',
 'tng, kai xin',
 'sean russell tay',
 'ryan suffiean bin rohaizat',
 'low, russell',
 'irfan qabeel mohamman',
 'huang, weijun',
 'harry irfan curran',
 'bharadwaj, akash vijay',
 'yeo shin kee (yang xuanqi)',
 'wang qiyue',
 'thaddaeus lee sheng en',
 'teow, alphonsus',
 'teo, yan',
 'tan, keane',
 'tan ying tong, shannon',
 'philip hoare, jonathan',
 'ng, caitlin shan wen',
 'meyapan, solaiy',
 'jamie el-red ang',
 'huang, weijun',
 'gan, ian',
 'fu zixuan',
 'b

In [1878]:
print(dictionary_dob.get("teow, alphonsus"))

1999-09-19 00:00:00


In [1634]:
final_names

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,NO_DOB
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,NaT,NaN,NaN,False
100,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,0.5626,0.76705,0.9715,0.29,7.127660,Tier 1,NaT,NaN,NaN,False
151,4356,"Medina, Andrew George",15.13,None,None,5,None,Triple Jump,None,None,...,1.7726,1.97705,2.1815,1.50,16.005136,Tier 1,NaT,NaN,NaN,False
365,12819,"Loh Ding Rong, Anson Ding Rong",17.2,None,None,1,None,Shot Put,None,5kg,...,0.8438,1.09415,1.3445,0.51,8.055722,Tier 1,NaT,NaN,NaN,False
535,14181,"Foo Ming Zhe, Sean",3.85,RI,16,1.0,B,Pole Vault,None,None,...,-0.0994,-0.03895,0.0215,-0.18,0.533499,Tier 4,NaT,NaN,16.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7700,17142,"Ng, Caitlin Shan Wen",00:12.45,SJI(INT),16,2.0,B,100m,None,None,...,0.0450,0.22875,0.4125,-0.20,3.367347,Tier 2,NaT,NaN,16.0,False
8117,5081,Joshua Shyen Lee,11.18,None,None,2,None,100m,None,None,...,-0.1130,0.04975,0.2125,-0.33,1.958525,Tier 3,NaT,NaN,NaN,False
8167,2847,"Huang, Weijun",11.1,None,None,4,None,100m,None,None,...,-0.0330,0.12975,0.2925,-0.25,2.695853,Tier 3,NaT,NaN,NaN,False
8184,8441,"Ho, Xander",11.02,Nanyang Technological Universi,12,3,Open,Dash,100,None,...,0.0470,0.20975,0.3725,-0.17,3.433180,Tier 2,NaT,NaN,NaN,False


In [1642]:
final_df[final_df['NAME']=='Loh Ding Rong, Anson Ding Rong']

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
365,12819,"Loh Ding Rong, Anson Ding Rong",17.2,None,None,1,None,Shot Put,None,5kg,...,17.2,0.8438,1.09415,1.3445,0.51,8.055722,Tier 1,NaT,NaN,NaN


In [1640]:
result = final_df.query('DOB_new.notnull() and NAME == "Loh Ding Rong, Anson Ding Rong"')


In [1641]:
result

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract


In [1622]:
# Iterate over selection to create dictionary of lists for DOB

#names = final_selection['NAME'].to_list()



for index, row in final_selection.iterrows():
    final_selection['GOT_DOB'] = final_selection['DOB'].isnull()
    
#    for col in df.columns:
#        if col not in dict_of_lists:
#            dict_of_lists[col] = []
#        dict_of_lists[col].append(row[col])

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_51640/3590391132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_selection['NO_DOB'] = final_selection['DOB'].isnull()


In [1623]:
final_selection

,index,NAME,RESULT_x,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,NO_DOB
47,12316,Tse Teng Tan,11.59,None,None,None,None,Triple Jump,None,None,...,-0.0524,0.12580,0.3040,-0.29,2.558923,Tier 3,NaT,NaN,NaN,False
57,6870,"Tang Kai Sheng, Cayman",14.17m,VICTORIA SCHOOL,16,1,U18,Triple Jump,0,None,...,0.8126,1.01705,1.2215,0.54,8.961849,Tier 1,2009-07-15,2009.0,16.0,False
100,804,"Saw, Xiang Yu",13.92m,Singapore Institute of Managem,12,2,Open,Triple Jump,0,None,...,0.5626,0.76705,0.9715,0.29,7.127660,Tier 1,NaT,NaN,NaN,False
151,4356,"Medina, Andrew George",15.13,None,None,5,None,Triple Jump,None,None,...,1.7726,1.97705,2.1815,1.50,16.005136,Tier 1,NaT,NaN,NaN,False
365,12819,"Loh Ding Rong, Anson Ding Rong",17.2,None,None,1,None,Shot Put,None,5kg,...,0.8438,1.09415,1.3445,0.51,8.055722,Tier 1,NaT,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8167,2847,"Huang, Weijun",11.1,None,None,4,None,100m,None,None,...,-0.0330,0.12975,0.2925,-0.25,2.695853,Tier 3,NaT,NaN,NaN,False
8184,8441,"Ho, Xander",11.02,Nanyang Technological Universi,12,3,Open,Dash,100,None,...,0.0470,0.20975,0.3725,-0.17,3.433180,Tier 2,NaT,NaN,NaN,False
8213,3343,Hakim Muhammad Aryan,11.23,None,None,1,None,100m,None,None,...,-0.1630,-0.00025,0.1625,-0.38,1.497696,Tier 4,2010-06-17,2010.0,15.0,False
8258,2126,"Fang, Kayden",11.37,VICTORIA SCHOOL,16,4,U18,Dash,100,None,...,-0.3030,-0.14025,0.0225,-0.52,0.207373,Tier 4,2009-02-19,2009.0,16.0,False


In [108]:
# Rank everyone for published ranking lists

published_ranking = final_df.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
published_ranking['Rank'] = published_ranking.groupby(['GENDER', 'MAPPED_EVENT']).cumcount() + 1

published_ranking.to_csv('published_ranking_prod.csv', encoding='utf-8')

In [109]:
# Rank everyone for octc selection

all_ranking = final_df.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
all_ranking['Rank'] = all_ranking.groupby(['GENDER', 'MAPPED_EVENT', 'TIER']).cumcount() + 1


In [565]:
# Convert time format for marathon and 5000m into mm:ss.00
# Choose the correct column indices or you will get erratic timings

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

all_ranking=all_ranking.reset_index(drop=True)


#all_ranking[['2%', '3.5%', '5%']] = df[['2%', '3.5%', '5%']].apply(pd.to_numeric)


#all_ranking['2%'] = all_ranking['2%'].astype("string")
#all_ranking['3.5%'] = all_ranking['3.5%'].astype("string")
#all_ranking['5%'] = all_ranking['5%'].astype("string")


for i in range(len(all_ranking)):
        
    rowIndex = all_ranking.index[i]

    event=all_ranking.iloc[rowIndex,21]
        
    
    time_base2=all_ranking.iloc[rowIndex,25]
    time_base3=all_ranking.iloc[rowIndex,26]
    time_base5=all_ranking.iloc[rowIndex,27]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10,000m' or event=='5000m' or event=='3000m Steeplechase' or event=='1500m':
        
      #  print(i, event, time_base2, time_base3, time_base5)

        
        

            
        
        date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
        date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
        date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)
        
    #    print(date_preconvert2, date_preconvert3, date_preconvert5)
            
        
        output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
        output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
        output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
     #   print(event, output2, output3, output5)

                    
       #     top_performers_clean.loc[rowIndex, '2%_timing'] = output2
       #     top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
       #     top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
   
        all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
        all_ranking.at[rowIndex, '3.5%'] = output3
        all_ranking.at[rowIndex, '5%'] = output5


            


        
    elif event=='Marathon':
        
      #  print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
        #    top_performers_clean.loc[rowIndex, '2%_timing'] = output2
        #    top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
        #    top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
            all_ranking.at[rowIndex, '3.5%'] = output3
            all_ranking.at[rowIndex, '5%'] = output5

            
         #   print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             


In [566]:
all_ranking

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
0,32796,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.5195,0.755,-0.03,4.808917,Tier 2,2003-02-23,2003.0,22.0,1
1,32799,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.044,0.2795,0.515,-0.27,3.280255,Tier 2,NaT,NaN,NaN,2
2,33352,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.1205,0.115,-0.67,0.732484,Tier 4,2003-08-25,2003.0,22.0,1
3,32842,Tan Shou Yi Rei (Chen Shouyi),14.99m,Raffles Institution JC,17,1,U20,Triple Jump,0,None,...,-0.396,-0.1605,0.075,-0.71,0.477707,Tier 4,2008-12-05,2008.0,17.0,2
4,33306,"Sueto, Yuito",14.76m,Kansai University of Social We,12,3,Open,Triple Jump,0,None,...,-0.626,-0.3905,-0.155,-0.94,-0.987261,,2004-01-06,2004.0,21.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18390,16467,"., Rasyiqah",31.51,Seng Kang Primary School,10,30,U11,Dash,150,,...,NaN,NaN,NaN,NaN,NaN,,2014-01-20,2014.0,11.0,1728
18391,11025,"., Nur Elena",16.42,UNA,6,1,U7,Dash,80,,...,NaN,NaN,NaN,NaN,NaN,,2018-12-04,2018.0,7.0,1729
18392,16747,"., Nur Amelia",16.64,Seng Kang Primary School,10,63,U11,Dash,80,,...,NaN,NaN,NaN,NaN,NaN,,2014-03-12,2014.0,11.0,1730
18393,26372,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,...,NaN,NaN,NaN,NaN,NaN,,2010-04-06,2010.0,15.0,1731


## Apply Rule E - Two Athletes per Tier, 3rd placing and below move down one tier

In [567]:
# Apply Rule E, two per tier only, therefore move 3rd and 4th placings down one tier

#all_ranking['TIER_ADJ'] = np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==4)), 'Tier2',
#                                np.where(
#                                ((all_ranking['TIER']=='Tier2') & (all_ranking['Rank']==3)), 'Tier3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier2') & (all_ranking['Rank']==4)), 'Tier3', ' ')
#                                
#                                )))


#all_ranking['TIER_ADJ'] = np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==4)), 'Tier 2',
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==3)), 'Tier 3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==4)), 'Tier 3', 
#                                np.where(
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==5)), 'Tier 3', 
#                                np.where(  
#                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==6)), 'Tier 3', 
#                                np.where(    
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==3)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==4)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==5)), 'Tier 4', 
#                                np.where(                             
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==6)), 'Tier 4', 
#                                np.where(                             
#                                
#                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==4)), 'Tier 4', all_ranking['TIER'])
#                                
#                               ))))))))))

all_ranking['TIER_ADJ'] = np.where(
                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']==3)), 'Tier 2',    
                                np.where(
                                ((all_ranking['TIER']=='Tier 1') & (all_ranking['Rank']>=4)), 'Tier 2',
                                np.where(
                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']==3)), 'Tier 3', 
                                np.where(
                                ((all_ranking['TIER']=='Tier 2') & (all_ranking['Rank']>=4)), 'Tier 3', 
                                np.where(                             
                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']==3)), 'Tier 4', 
                                np.where(                             
                                ((all_ranking['TIER']=='Tier 3') & (all_ranking['Rank']>=4)), 'Tier 4', all_ranking['TIER']) 
                
                                )))))



In [568]:
all_ranking.to_csv('octc_all_ranking.csv', encoding='utf-8')

In [569]:
# Re-rank based on MAPPED_EVENT, GENDER, TIER_ADJ & PERF_SCALAR

rerank = all_ranking.sort_values(['MAPPED_EVENT','GENDER','TIER_ADJ', 'PERF_SCALAR'], ascending=[False, False, False, False])
rerank['Rank_ADJ'] = rerank.groupby(['MAPPED_EVENT', 'GENDER', 'TIER_ADJ']).cumcount() + 1


In [570]:
rerank_filtered=rerank[(rerank['TIER_ADJ']!=' ') & (rerank['TIER_ADJ']!='Tier 4')]

In [571]:
rerank_filtered

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank,TIER_ADJ,Rank_ADJ
0,32796,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.7550,-0.03,4.808917,Tier 2,2003-02-23,2003.0,22.0,1,Tier 2,1
1,32799,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.5150,-0.27,3.280255,Tier 2,NaT,NaN,NaN,2,Tier 2,2
230,32761,"ROZARIO, TIA LOUISE",13.01,None,None,1,None,Triple Jump,None,None,...,0.2230,-0.45,1.656761,Tier 3,2000-10-14,2000.0,25.0,1,Tier 3,1
559,25935,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.4000,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1,Tier 1,1
751,25763,"CHAN, JEREMY WEI LUN",2:40:21,INDIVIDUAL,35,1064,None,Marathon,None,None,...,195.4500,-272.00,2.090598,Tier 3,1989-05-05,1989.0,36.0,1,Tier 3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12682,17333,"PEREIRA, VERONICA SHANTI",11.45,Singapore,12,1,Open,Dash,100,None,...,0.8875,0.30,7.553191,Tier 1,1996-09-20,1996.0,29.0,1,Tier 1,1
12683,17438,"TAN, ELIZABETH-ANN",11.73,Singapore,12,3,Open,Dash,100,None,...,0.6075,0.02,5.170213,Tier 1,2003-09-23,2003.0,22.0,2,Tier 1,2
14149,5,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,...,108.3750,14.10,5.747812,Tier 1,1991-05-05,1991.0,34.0,1,Tier 1,1
14198,2,Chui Ling Goh,36:29.7,None,None,8,None,"10,000m",None,None,...,47.8500,-58.70,2.245425,Tier 3,1992-11-27,1992.0,33.0,1,Tier 3,1


In [572]:
# Check this output to ensure every new record upon upload has an event mapping and perf_scalar calc

rerank_filtered.to_csv('octc_rerank_filtered.csv', encoding='utf-8')

In [782]:
tuple = list(zip(rerank_filtered['NAME'], rerank_filtered['MAPPED_EVENT']))

In [783]:
tuple

[('LEE GABRIEL JIN YI', 'Triple Jump'),
 ('MEDINA, ANDREW GEORGE', 'Triple Jump'),
 ('ROZARIO, TIA LOUISE', 'Triple Jump'),
 ('LOW, JUN YU', 'Pole Vault'),
 ('CHAN, JEREMY WEI LUN', 'Marathon'),
 ('DANIEL LEOW SOON YE', 'Marathon'),
 ('He Yong', 'Marathon'),
 ('Yaohan Melvin Wong', 'Marathon'),
 ('TAN, AARON JUSTIN WEN JIE', 'Marathon'),
 ('Wei Xiang Gordon Lim', 'Marathon'),
 ('SOH RUI YONG GUILLAUME', 'Marathon'),
 ('HIA, CALEB', 'Marathon'),
 ('GOH SHING LING', 'Marathon'),
 ('MEDINA, ANDREW', 'Long Jump'),
 ('EMERY, CONRAD', 'Long Jump'),
 ('ROZARIO, TIA LOUISE', 'Long Jump'),
 ('CHLOE CHEE EN-YA', 'Long Jump'),
 ('Kampton Kam', 'High Jump'),
 ('GOH, AMELIA', 'High Jump'),
 ('MICHELLE SNG', 'High Jump'),
 ('YEE, CHUN WAI, ERIC', 'Discus Throw'),
 ('Lee, Tzu Yun', 'Discus Throw'),
 ('Shawn, Chia', '800m'),
 ('Fiore, Oliver', '800m'),
 ('nan', '800m'),
 ('ZUBIN PERCY MUNCHERJI', '800m'),
 ('JIE CONG JAYDEN, TAN', '800m'),
 ('LIM, OLIVER', '800m'),
 ('THANA RAJAN, THIRUBEN  S/O', '800

In [784]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

mar25 = pd.read_csv("rerank_filtered_mar25.csv")

In [785]:
mar25

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,...,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER_ADJ,Rank_ADJ
0,LEE GABRIEL JIN YI,15.67m,NUS,12.0,1.0,Open,0.0,NaN,L435B06,23/2/03,...,14.915,15.67,15.67,0.2840,0.51950,0.7550,-0.03,4.808917,Tier 2,1
1,"MEDINA, ANDREW GEORGE",15.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,...,14.915,15.43,15.43,0.0440,0.27950,0.5150,-0.27,3.280255,Tier 2,2
2,"ROZARIO, TIA LOUISE",13.01,NaN,NaN,1.0,NaN,NaN,NaN,NaN,14-Oct-00,...,12.787,13.01,13.01,-0.1808,0.02110,0.2230,-0.45,1.656761,Tier 3,1
3,Anson Loh Ding Rong,17.2,NaN,NaN,1.0,NaN,NaN,5kg,NaN,8,...,16.435,17.20,17.20,0.2460,0.50550,0.7650,-0.10,4.421965,Tier 2,1
4,"LOW, JUN YU",5.34,NaN,NaN,1.0,NaN,NaN,NaN,NaN,21-Apr-01,...,4.94,5.34,5.34,0.2440,0.32200,0.4000,0.14,7.692308,Tier 1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12.0,1.0,Open,100.0,NaN,NaN,23/9/03,...,12.3375,11.99,11.99,-0.0050,0.17125,0.3475,-0.24,2.957447,Tier 3,1
65,"JAIGANTH, LAAVINIA",12.15,Ngee Ann Polytechnic,12.0,1.0,Open,100.0,NaN,L435B06,22/1/06,...,12.3375,12.15,12.15,-0.1650,0.01125,0.1875,-0.40,1.595745,Tier 3,2
66,"PEREIRA, VERONICA SHANTI",11.47,NaN,NaN,1.0,NaN,NaN,NaN,NaN,20-Sep-96,...,12.3375,11.47,11.47,0.5150,0.69125,0.8675,0.28,7.382979,Tier 1,1
67,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33.0,133.0,NaN,NaN,NaN,NaN,1991,...,32:59.8,1871.40,1871.40,51.8100,80.09250,108.3750,14.10,5.747812,Tier 1,1


In [3244]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

dec24 = pd.read_csv("octc_rule_2d_cut_dec24.csv")
mar25 = pd.read_csv("octc_rule_2d_cut_mar25.csv")

In [3245]:
compare = datacompy.Compare(
    dec24,
    mar25,
    join_columns=['NAME', 'MAPPED_EVENT'],  #You can also specify a list of columns
    rel_tol=0, #Optional, defaults to 0
    )

In [3246]:
compare.matches(ignore_extra_columns=False)

print(compare.report())


DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1        5    50
1       df2        5    17

Column Summary
--------------

Number of columns in common: 5
Number of columns in df1 but not in df2: 0 []
Number of columns in df2 but not in df1: 0 []

Row Summary
-----------

Matched on: name, mapped_event
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 9
Number of rows in df1 but not in df2: 41
Number of rows in df2 but not in df1: 8

Number of rows with some compared columns unequal: 7
Number of rows with all compared columns equal: 2

Column Comparison
-----------------

Number of columns compared with some values unequal: 3
Number of columns compared with all values equal: 2
Total number of values which compare unequal: 14

Columns with Unequal Values or Types
------------------------------------

        Column df1 dtype df2 dtype  # Unequal  Max Diff  # Null D

In [2814]:
print(compare.df2_unq_rows)

                     name  mapped_event  result_best tier_adj  final_rank
2        CHLOE CHEE EN-YA     Long Jump         5.90   Tier 2         2.0
3           Chua, Garrett  110m Hurdles        14.25   Tier 3         3.0
16              Jayden Ng     Decathlon      5716.00   Tier 1         1.0
31            Low, Nicole         5000m      1041.30   Tier 2         2.0
51        Tak Yeung Leung         5000m       903.90   Tier 3         2.0
53          Tate Tan Fung          100m        10.50   Tier 2         2.0
54  Thiruben, Thana Rajan          800m       113.70   Tier 2         2.0
55             YAN, ETHAN         1500m       243.90   Tier 3         3.0


## Apply Rule 3 - If athlete qualifies for more than one event, the higher tier event is selected

In [2141]:
octc_rule_3 = rerank_filtered.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1)

In [2142]:
octc_rule_3

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank,TIER_ADJ,Rank_ADJ
5570,25335,"{'545': 'Zulkhair, Muhd', '546': 'TAN, DARYL',...",41.06,Team Cicada Trackers,12,1,Open,Relay,400,,...,0.2680,-1.70,0.680894,Tier 4,NaT,NaN,NaN,2,Tier 4,2
5263,25248,"{'442': 'Chauhan, Aarya', '455': 'RYAN, PRAHAR...",03:18.1,Oldham Athletics,12,1,Open,Relay,1600,,...,0.1400,-9.30,0.074153,Tier 4,NaT,NaN,NaN,1,Tier 4,1
5689,25077,"{'183': 'LEE, MARK REN', '159': 'RYAN, PRAHARS...",40.88,SINGAPORE,12,1,Open,Relay,400,,...,0.4480,-1.52,1.138211,Tier 4,NaT,NaN,NaN,1,Tier 4,1
5712,5158,"{'135': 'Koe, Ian', '136': 'GAN, IAN', '139': ...",41.27,Trackstar Athletics,12,1,Open,Relay,400,None,...,0.0580,-1.91,0.147358,Tier 4,2015-04-19,2015.0,10.0,4,Tier 4,4
5717,4689,"{'1335': 'TAN, DARYL HONG AN', '1336': 'Teo, Y...",41.1,Singapore,12,1,Open,Relay,400,Trial,...,0.2280,-1.74,0.579268,Tier 4,2010-03-21,2010.0,15.0,3,Tier 4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3562,1157,"Aswin, Kumar",01:54.9,Oldham Athletics,19,1,U20,Run,800,None,...,4.6950,-1.00,4.122037,Tier 2,2006-04-13,2006.0,19.0,2,Tier 2,3
10072,23506,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,0.1625,-0.36,1.555024,Tier 3,2005-07-02,2005.0,20.0,10,Tier 4,8
498,7059,Anson Loh Ding Rong,17.2,None,None,1,None,Shot Put,None,5kg,...,0.7650,-0.10,4.421965,Tier 2,NaT,NaN,NaN,1,Tier 2,1
8319,27741,"Adarsh, Aravinth",04:07.1,National University Singapore,12,2,Open,Run,1500,None,...,4.2700,-7.70,1.783626,Tier 3,2001-10-13,2001.0,24.0,4,Tier 4,2


In [2143]:
octc_rule_3.to_csv('octc_rule_3.csv', encoding='utf-8')

## Apply Rule 2d - max 3 per event unless 100m/400m then it is max 6

In [2796]:
#octc_rule_2d = octc_rule_3[(   
#    ((octc_rule_3['MAPPED_EVENT']=='400m')|(octc_rule_3['MAPPED_EVENT']=='100m'))&((octc_rule_3['TIER_ADJ']=='Tier 1')|(octc_rule_3['TIER_ADJ']=='Tier 2')|(octc_rule_3['TIER_ADJ']=='Tier 3'))&((octc_rule_3['Rank_ADJ']==1)|(octc_rule_3['Rank_ADJ']==2))
#    |
#    (
#        ~((octc_rule_3['MAPPED_EVENT']=='400m')|(octc_rule_3['MAPPED_EVENT']=='100m'))&((octc_rule_3['TIER_ADJ']=='Tier 1')|(octc_rule_3['TIER_ADJ']=='Tier 2')|(octc_rule_3['TIER_ADJ']=='Tier 3'))&((octc_rule_3['Rank_ADJ']==1)|(octc_rule_3['Rank_ADJ']==2))
#    )
#    )]

octc_rule_2d = rerank_filtered[(   
    ((rerank_filtered['MAPPED_EVENT']=='400m')|(rerank_filtered['MAPPED_EVENT']=='100m'))&((rerank_filtered['TIER_ADJ']=='Tier 1')|(rerank_filtered['TIER_ADJ']=='Tier 2')|(rerank_filtered['TIER_ADJ']=='Tier 3'))&((rerank_filtered['Rank_ADJ']==1)|(rerank_filtered['Rank_ADJ']==2))
    |
    (
        ~((rerank_filtered['MAPPED_EVENT']=='400m')|(rerank_filtered['MAPPED_EVENT']=='100m'))&((rerank_filtered['TIER_ADJ']=='Tier 1')|(rerank_filtered['TIER_ADJ']=='Tier 2')|(rerank_filtered['TIER_ADJ']=='Tier 3'))&((rerank_filtered['Rank_ADJ']==1)|(rerank_filtered['Rank_ADJ']==2))
    )
    )]



In [2797]:
octc_rule_2d.to_csv('octc_rule_2d_prelim.csv', encoding='utf-8')

In [2798]:
# Rank again so we can choose top 6 for 100m/400m and top 3 for all other events

octc_rule_2d = octc_rule_2d.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
octc_rule_2d['Final_Rank'] = octc_rule_2d.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1


In [2799]:
octc_rule_2d

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank,TIER_ADJ,Rank_ADJ,Final_Rank
0,23997,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,-0.03,4.808917,Tier 2,2003-02-23,2003.0,22.0,1,Tier 2,1,1
1,6986,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,-0.27,3.280255,Tier 2,NaT,NaN,NaN,2,Tier 2,2,2
203,6945,"ROZARIO, TIA LOUISE",13.01,None,None,1,None,Triple Jump,None,None,...,-0.45,1.656761,Tier 3,2000-10-14,2000.0,25.0,1,Tier 3,1,1
404,7059,Anson Loh Ding Rong,17.2,None,None,1,None,Shot Put,None,5kg,...,-0.10,4.421965,Tier 2,NaT,NaN,NaN,1,Tier 2,1,1
502,7035,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1,Tier 1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9102,6924,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1,Tier 1,1,1
9103,24052,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,-0.24,2.957447,Tier 3,2003-09-23,2003.0,22.0,1,Tier 3,1,2
9104,22622,"JAIGANTH, LAAVINIA",12.15,Ngee Ann Polytechnic,12,1,Open,Dash,100,,...,-0.40,1.595745,Tier 3,2006-01-22,2006.0,19.0,2,Tier 3,2,3
9759,6000,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,...,14.10,5.747812,Tier 1,1991-04-10,1991.0,34.0,1,Tier 1,1,1


In [2800]:
# Choose 3/6 for each event

octc_rule_2d = octc_rule_2d[(((octc_rule_2d['MAPPED_EVENT']=='400m')|(octc_rule_2d['MAPPED_EVENT']=='100m'))&(octc_rule_2d['Final_Rank']<7))|(~((octc_rule_2d['MAPPED_EVENT']=='400m')|(octc_rule_2d['MAPPED_EVENT']=='100m'))&(octc_rule_2d['Final_Rank']<4))]


In [2801]:
octc_rule_2d.to_csv('octc_rule_2d_final.csv', encoding='utf-8')

# Top 8 Performers

In [820]:
# Filter top 8 performances for each event and gender

top_eight_performers = tiered_performers.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'],ascending=[False, False, False]).groupby(['GENDER', 'MAPPED_EVENT']).head(8).reset_index(drop=True)

top_eight_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
0,23028,LEE GABRIEL JIN YI,15.67m,NUS,21,1,Open,Triple Jump,0,,...,15.1505,14.915,15.67,15.67,0.284,0.5195,0.755,-0.03,4.808917,Tier2
1,5676,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,15.1505,14.915,15.43,15.43,0.044,0.2795,0.515,-0.27,3.280255,Tier2
2,23153,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,21,1,Open,Triple Jump,0,,...,15.1505,14.915,15.03,15.03,-0.356,-0.1205,0.115,-0.67,0.732484,Tier4
3,912,"Kumar, Viresh",14.36m,Victoria Junior College,18,1,U20,Triple Jump,0,None,...,15.1505,14.915,14.36,14.36,-1.026,-0.7905,-0.555,-1.34,-3.535032,
4,5838,Tan Shou Yi Rei (Chen Shouyi),14.12,None,None,1,None,Triple Jump,None,None,...,15.1505,14.915,14.12,14.12,-1.266,-1.0305,-0.795,-1.58,-5.063694,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,3678,"{'90': 'CHEW, CAERISSE KAI YI', '1438': 'TNG, ...",12:22.0,Dunman High School,10,2,B Division,Relay,3200,None,...,NaN,NaN,742.0,742.00,NaN,NaN,NaN,NaN,NaN,
451,3639,"{'879': 'WONG YU XIM, ANDREA', '884': 'YONG, S...",01:54.3,Nanyang Girls' High School,10,2,C Division,Relay,800,None,...,NaN,NaN,114.3,114.30,NaN,NaN,NaN,NaN,NaN,
452,3654,"{'870': 'PUAR MIN, QUINN', '872': 'CHONG KATE ...",01:53.3,Nanyang Girls' High School,10,1,B Division,Relay,800,None,...,NaN,NaN,113.3,113.30,NaN,NaN,NaN,NaN,NaN,
453,3491,"{'813': 'LEA, PANG SUI ON', '814': 'CHARYNE, C...",37.74,CHIJ Primary (Toa Payoh),10,19,Junior 2,Relay,200,None,...,NaN,NaN,37.74,37.74,NaN,NaN,NaN,NaN,NaN,


In [831]:
top_eight_performers.to_csv('top_8_prod.csv', encoding='utf-8')

# Apply OCTC Rules (OLD)

## OCTC Rule 4 - if athlete qualifies in 2 events, choose the better performing one

In [822]:
# Change to numeric

final_df[['2%', '3.5%', '5%', 'RESULT_CONV']] = final_df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/4012151466.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['2%', '3.5%', '5%', 'RESULT_CONV']] = final_df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)


In [823]:
# Select qualifiers based on 5% tolerance

#octc_rule3 = octc_rule4.loc[(((octc_rule4['CATEGORY_EVENT']=='Mid')|(octc_rule4['CATEGORY_EVENT']=='Sprint')|(octc_rule4['CATEGORY_EVENT']=='Long')|(octc_rule4['CATEGORY_EVENT']=='Hurdles')|(octc_rule4['CATEGORY_EVENT']=='Walk')|(octc_rule4['CATEGORY_EVENT']=='Relay')|(octc_rule4['CATEGORY_EVENT']=='Marathon')|(octc_rule4['CATEGORY_EVENT']=='Steeple')|(octc_rule4['CATEGORY_EVENT']=='Pentathlon')|(octc_rule4['CATEGORY_EVENT']=='Heptathlon')|(octc_rule4['CATEGORY_EVENT']=='Triathlon'))&(octc_rule4['RESULT_CONV'] <= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))|(((octc_rule4['CATEGORY_EVENT']=='Jump')|(octc_rule4['CATEGORY_EVENT']=='Throw'))&(octc_rule4['RESULT_CONV'] >= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))]

octc_rule4 = final_df.loc[
    (
    ((final_df['CATEGORY_EVENT']=='Mid')|(final_df['CATEGORY_EVENT']=='Sprint')|(final_df['CATEGORY_EVENT']=='Long')|(final_df['CATEGORY_EVENT']=='Hurdles')|(final_df['CATEGORY_EVENT']=='Walk')|(final_df['CATEGORY_EVENT']=='Relay')|(final_df['CATEGORY_EVENT']=='Marathon')|(final_df['CATEGORY_EVENT']=='Steeple')|(final_df['CATEGORY_EVENT']=='Pentathlon')|(final_df['CATEGORY_EVENT']=='Heptathlon')|(final_df['CATEGORY_EVENT']=='Triathlon'))
    
    & (final_df['RESULT_CONV'] <= final_df['5%']) 
    
#    & (octc_rule4['age_extract']<40) 
    
#    & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon'))
    )  
    |(
        ((final_df['CATEGORY_EVENT']=='Jump')|(final_df['CATEGORY_EVENT']=='Throw'))
        
    & (final_df['RESULT_CONV'] >= final_df['5%']) 
#    & (octc_rule4['age_extract']<40) 
#    & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon'))
    )
    ]

In [824]:
octc_rule4.to_csv('octc_rule4_prod.csv', encoding='utf-8')

In [825]:
#top_performers_final = top_performers_sorted.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, False], na_position='last')
octc_rule4_final = octc_rule4.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1)

In [826]:
# Rank list of athletes by group after sorting

#octc_rule3['Rank'] = (octc_rule3.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, True])['MAPPED_EVENT', 'GENDER']
#                .rank(method='first', ascending=False)
#             )


#octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'])
#octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1

octc_rule4_final = octc_rule4_final.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
octc_rule4_final['Rank'] = octc_rule4_final.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1



In [827]:
octc_rule4_final

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
234,23028,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.51950,0.7550,-0.03,4.808917,Tier2,2003-02-23,2003.0,22.0,1
173,5676,"MEDINA, ANDREW GEORGE",15.43,None,None,None,None,Triple Jump,None,None,...,0.044,0.27950,0.5150,-0.27,3.280255,Tier2,NaT,NaN,NaN,2
248,23153,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.12050,0.1150,-0.67,0.732484,Tier4,2003-08-25,2003.0,22.0,3
539,5725,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.244,0.32200,0.4000,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1
625,5655,SOH RUI YONG GUILLAUME,2:24:07,None,None,41,None,Marathon,None,None,...,888.980,1029.21500,1169.4500,702.00,12.508824,Tier 1,1991-08-06,1991.0,34.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10270,1695,"Ng, Bryan",10.95,Wings Athletics Club,12,5,Open,Dash,100,None,...,-0.291,-0.13425,0.0225,-0.50,0.215311,Tier4,2000-10-11,2000.0,25.0,22
10652,177,"Lee Shyen, Joshua",10.96,TeamFabian,17,1,U18,Dash,100,None,...,-0.301,-0.14425,0.0125,-0.51,0.119617,Tier4,2008-09-12,2008.0,17.0,23
10162,5614,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.515,0.69125,0.8675,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1
9881,23083,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,-0.005,0.17125,0.3475,-0.24,2.957447,Tier3,2003-09-23,2003.0,22.0,2


In [828]:
octc_rule4_final.to_csv('octc_rule4_final.csv', encoding='utf-8')

## OCTC Rule 3 - Where top athlete is >30 yrs old (except marathon), to include next athlete as well (below 30)

In [257]:
# Apply Rule 3 & is within 5% performance band

octc_rule3 = octc_rule4_final.loc[(((octc_rule4['CATEGORY_EVENT']=='Mid')|(octc_rule4['CATEGORY_EVENT']=='Sprint')|(octc_rule4['CATEGORY_EVENT']=='Long')|(octc_rule4['CATEGORY_EVENT']=='Hurdles')|(octc_rule4['CATEGORY_EVENT']=='Walk')|(octc_rule4['CATEGORY_EVENT']=='Relay')|(octc_rule4['CATEGORY_EVENT']=='Marathon')|(octc_rule4['CATEGORY_EVENT']=='Steeple')|(octc_rule4['CATEGORY_EVENT']=='Pentathlon')|(octc_rule4['CATEGORY_EVENT']=='Heptathlon')|(octc_rule4['CATEGORY_EVENT']=='Triathlon'))&(octc_rule4['RESULT_CONV'] <= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))|(((octc_rule4['CATEGORY_EVENT']=='Jump')|(octc_rule4['CATEGORY_EVENT']=='Throw'))&(octc_rule4['RESULT_CONV'] >= octc_rule4['5%']) & (octc_rule4['age_extract']<40) & ((octc_rule4['MAPPED_EVENT']!='Marathon')|(octc_rule4['age_extract']<60) & (octc_rule4['MAPPED_EVENT']=='Marathon')))]

In [258]:
octc_rule3

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
2570,1607,nan,01:56.9,None,None,2,None,800m,None,None,...,116.90,-0.7220,0.98650,2.6950,-3.00,2.366111,Tier3,2006-06-24,2006.0,19.0
2580,1606,ZUBIN PERCY MUNCHERJI,01:53.7,None,None,2,None,800m,None,None,...,113.70,2.4780,4.18650,5.8950,0.20,5.175593,Tier 1,1996-06-23,1996.0,29.0
2433,20042,"YEE, CHUN WAI, ERIC",49.51m,Hwa Chong Alumni Association,12,1,Open,Discus Throw,0,(2kg),...,49.51,0.4904,1.24070,1.9910,-0.51,3.980408,Tier2,1999-03-01,1999.0,26.0
540,1600,Wei Xiang Gordon Lim,2:36:19,None,None,566,None,Marathon,None,None,...,9379.00,156.9800,297.21500,437.4500,-30.00,4.679110,Tier2,1993-02-07,1993.0,32.0
8604,22740,"Toh Jun Xi, Tedd",10.93,National University Singapore,12,4,Open,Dash,100,None,...,10.93,-0.2710,-0.11425,0.0425,-0.48,0.406699,Tier4,2002-03-18,2002.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10273,2816,Brayden Chan Wei Jie,00:10.86,RI,18.5,3,A,100m,None,None,...,10.86,-0.2010,-0.04425,0.1125,-0.41,1.076555,Tier4,NaT,NaN,18.5
3345,17663,"Aswin, Kumar",01:56.0,Temasek Polytechnic,12,4,Open,Run,800,,...,116.00,0.1780,1.88650,3.5950,-2.10,3.156277,Tier2,2006-04-13,2006.0,19.0
10347,18575,"Ashok, Tharun",10.81,Ngee Ann Polytechnic,12,4,Open,Dash,100,,...,10.81,-0.1510,0.00575,0.1625,-0.36,1.555024,Tier3,2005-07-02,2005.0,20.0
8286,22810,"Adarsh, Aravinth",04:07.1,National University Singapore,12,2,Open,Run,1500,None,...,247.10,-2.9120,0.67900,4.2700,-7.70,1.783626,Tier3,2001-10-13,2001.0,24.0


In [259]:
# Rank list of athletes by group after sorting

#octc_rule3['Rank'] = (octc_rule3.sort_values(by=['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[False, False, True])['MAPPED_EVENT', 'GENDER']
#                .rank(method='first', ascending=False)
#             )


#octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'])
#octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1

octc_rule3 = octc_rule3.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
octc_rule3['Rank'] = octc_rule3.groupby(['MAPPED_EVENT', 'GENDER']).cumcount() + 1



In [260]:
octc_rule3

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract,Rank
193,19066,LEE GABRIEL JIN YI,15.67m,NUS,12,1,Open,Triple Jump,0,,...,0.284,0.51950,0.7550,-0.03,4.808917,Tier2,2003-02-23,2003.0,22.0,1
206,19191,"Kapil, Arnav",15.03m,WINGS ATHLETICS CLUB,12,1,Open,Triple Jump,0,,...,-0.356,-0.12050,0.1150,-0.67,0.732484,Tier4,2003-08-25,2003.0,22.0,2
476,1763,"LOW, JUN YU",5.34,None,None,1,None,Pole Vault,None,None,...,0.244,0.32200,0.4000,0.14,7.692308,Tier 1,2001-04-21,2001.0,24.0,1
549,1693,SOH RUI YONG GUILLAUME,2:24:07,None,None,41,None,Marathon,None,None,...,888.980,1029.21500,1169.4500,702.00,12.508824,Tier 1,1991-08-06,1991.0,34.0,1
545,772,"TAN, AARON JUSTIN WEN JIE",2:35:26,INDIVIDUAL,31,None,None,Marathon,None,None,...,209.980,350.21500,490.4500,23.00,5.246016,Tier 1,1993-04-01,1993.0,32.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,21106,MARK LEE REN,10.92,INDIVIDUAL,21,None,None,100m,None,None,...,-0.261,-0.10425,0.0525,-0.47,0.502392,Tier4,2004-04-01,2004.0,21.0,17
8604,22740,"Toh Jun Xi, Tedd",10.93,National University Singapore,12,4,Open,Dash,100,None,...,-0.271,-0.11425,0.0425,-0.48,0.406699,Tier4,2002-03-18,2002.0,23.0,18
9091,1652,"PEREIRA, VERONICA SHANTI",11.47,None,None,1,None,100m,None,None,...,0.515,0.69125,0.8675,0.28,7.382979,Tier 1,1996-09-20,1996.0,29.0,1
8826,19121,"TAN, ELIZABETH-ANN",11.99,SINGAPORE,12,1,Open,Dash,100,,...,-0.005,0.17125,0.3475,-0.24,2.957447,Tier3,2003-09-23,2003.0,22.0,2


In [261]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

octc_rule3.to_csv('octc_rule3.csv', encoding='utf-8')

## OCTC Rule 6 - max 6 for 100m/400m and max 3 for all other events

In [829]:
octc_rule6 = octc_rule4_final[(((octc_rule4_final['MAPPED_EVENT']=='400m')|(octc_rule4_final['MAPPED_EVENT']=='100m'))&(octc_rule4_final['Rank']<7))|(~((octc_rule4_final['MAPPED_EVENT']=='400m')|(octc_rule4_final['MAPPED_EVENT']=='100m'))&(octc_rule4_final['Rank']<4))]


In [830]:
octc_rule6.to_csv('octc_rule6.csv', sep=',', encoding='utf-8-sig', index=False)


In [364]:
#rslt_df['Rank'] = (rslt_df.sort_values(by=['EVENT', 'GENDER', 'Delta35'], ascending=[False, False, True])['Delta35']
#                .rank(method='first', ascending=False)
#             )


# Download list of foreigners and spex carded athletes

In [365]:
#spex_list=pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/SPEX_CARDED_LIST.csv', encoding='latin-1')


In [1622]:
foreigners = pd.read_csv('/Users/veesheenyuen/Desktop/DataScience/SAA/MM/List of Foreigners.csv', encoding='latin-1')


In [1623]:
#spex_list

In [432]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN
1,Akahodani,Takayuki,NaN,NaN,NaN
2,Apondar,Audric,NaN,NaN,NaN
3,Brooks,Ruby,NaN,NaN,NaN
4,Brouwer,Cees,NaN,NaN,NaN
...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN
220,NGUYEN,HOAI VAN,NaN,NaN,NaN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN


In [433]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

In [434]:
foreigners

,LAST_NAME,FIRST_NAME,Unnamed: 2,Unnamed: 3,Unnamed: 4,V1,V2,V3,V4
0,Aaryan,Greuter Christoph,NaN,NaN,NaN,Aaryan Greuter Christoph,Greuter Christoph Aaryan,"Aaryan, Greuter Christoph",Greuter Christoph Aaryan
1,Akahodani,Takayuki,NaN,NaN,NaN,Akahodani Takayuki,Takayuki Akahodani,"Akahodani, Takayuki",Takayuki Akahodani
2,Apondar,Audric,NaN,NaN,NaN,Apondar Audric,Audric Apondar,"Apondar, Audric",Audric Apondar
3,Brooks,Ruby,NaN,NaN,NaN,Brooks Ruby,Ruby Brooks,"Brooks , Ruby",Ruby Brooks
4,Brouwer,Cees,NaN,NaN,NaN,Brouwer Cees,Cees Brouwer,"Brouwer, Cees",Cees Brouwer
...,...,...,...,...,...,...,...,...,...
219,CHO,CHIA-HSUAN,NaN,NaN,NaN,CHO CHIA-HSUAN,CHIA-HSUAN CHO,"CHO, CHIA-HSUAN",CHIA-HSUAN CHO
220,NGUYEN,HOAI VAN,NaN,NaN,NaN,NGUYEN HOAI VAN,HOAI VAN NGUYEN,"NGUYEN , HOAI VAN",HOAI VAN NGUYEN
221,NGUYEN,QUOC THINH,NaN,NaN,NaN,NGUYEN QUOC THINH,QUOC THINH NGUYEN,"NGUYEN , QUOC THINH",QUOC THINH NGUYEN
222,PHAM,QUYNH GIANG,NaN,NaN,NaN,PHAM QUYNH GIANG,QUYNH GIANG PHAM,"PHAM , QUYNH GIANG",QUYNH GIANG PHAM


In [371]:
#spex1 = spex_list['V1'].dropna().tolist()
#spex2 = spex_list['V2'].dropna().tolist()
#spex3 = spex_list['V3'].dropna().tolist()
#spex4 = spex_list['V4'].dropna().tolist()
#spex5 = spex_list['V5'].dropna().tolist()


In [435]:
for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()


In [373]:
#spex_athletes=spex1+spex2+spex3+spex4+spex5

In [436]:
foreign_list = for1+for2+for3+for4 

In [437]:
#spex_athletes

In [438]:
foreign_list

['Aaryan Greuter Christoph',
 'Akahodani Takayuki',
 'Apondar Audric',
 'Brooks  Ruby',
 'Brouwer Cees',
 'CARLESATER ARMAAN',
 'Chary Mathangi ',
 'Cinthikael Angel',
 'Comia Tsang Hannah',
 'Comia Tsang Mhandy',
 'Donnelly Jordan ',
 'Dorai Trinity Sharilyn',
 'Freeman Alexander',
 'Gaume Constance',
 'Gregorio John Alexander',
 'HAGIWARA RYOTA ',
 'Indarto Gerard',
 'Joy Matthew',
 'JULURI ADITYA',
 'Kiet Tran',
 'Koduru Kalyani',
 'Kuchenbuch Natalya',
 'KUIJPERS KUIJPERS AYKO ELLIN',
 'Kumar Joseph Suraj',
 'KUPPUSAMY PRAKSHEETA',
 'Kusumo Naia',
 'Kuwalekar  Spruha ',
 'Lahey Kezia Stephanie ',
 'Liz Jerry',
 'Marican Raees Marican Bin Ayoob Marican',
 'MARIE BRUNO DE FERRIERES DE SAUVEBOEUF LOUIS ',
 'Maslov Timofei',
 'Masrezwan Maeva',
 'MENDOZA NICCOLO',
 'Mennella Leonardo',
 'Meyers Nikita Mae Jiny-Yu',
 'Mishra Sunay',
 'Mo Thant Cin',
 'Moon Daniel Youngjo',
 'OCAMPO ANIAG DWAYNE MICHAEL OCAMPO ANIAG',
 'Ortega Rafael Pedro',
 'Owen Elia',
 'owl emma',
 'Ozbudak Deniz',
 

In [439]:
#spex_athletes_casefold=[s.casefold() for s in spex_athletes]

In [440]:
foreign_list_casefold=[s.casefold() for s in foreign_list]

In [442]:
#spex_athletes_casefold

In [443]:
foreign_list_casefold

['aaryan greuter christoph',
 'akahodani takayuki',
 'apondar audric',
 'brooks  ruby',
 'brouwer cees',
 'carlesater armaan',
 'chary mathangi ',
 'cinthikael angel',
 'comia tsang hannah',
 'comia tsang mhandy',
 'donnelly jordan ',
 'dorai trinity sharilyn',
 'freeman alexander',
 'gaume constance',
 'gregorio john alexander',
 'hagiwara ryota ',
 'indarto gerard',
 'joy matthew',
 'juluri aditya',
 'kiet tran',
 'koduru kalyani',
 'kuchenbuch natalya',
 'kuijpers kuijpers ayko ellin',
 'kumar joseph suraj',
 'kuppusamy praksheeta',
 'kusumo naia',
 'kuwalekar  spruha ',
 'lahey kezia stephanie ',
 'liz jerry',
 'marican raees marican bin ayoob marican',
 'marie bruno de ferrieres de sauveboeuf louis ',
 'maslov timofei',
 'masrezwan maeva',
 'mendoza niccolo',
 'mennella leonardo',
 'meyers nikita mae jiny-yu',
 'mishra sunay',
 'mo thant cin',
 'moon daniel youngjo',
 'ocampo aniag dwayne michael ocampo aniag',
 'ortega rafael pedro',
 'owen elia',
 'owl emma',
 'ozbudak deniz',
 

In [444]:
# Exclusion list is foreigners + spex carded 

#exclusions = foreign_list_casefold + spex_athletes_casefold

exclusions = foreign_list_casefold

In [445]:
# Apply Rule 4 above...if an athlete is in more than one event, choose the best performing one

top_performers=rslt_df.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [446]:
top_performers

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,EVENT_y,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR
8353,11670,{},12.99,8.0,2,Boys 4x100 Meter Relay 15-16,2015-01-01 00:00:00.000,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.99,27.1572,27.74760,28.3380,71.996951
12480,17949,"{9: 'Ng, Caitlin Shan Wen', 438: 'Seow, Kyra',...",22.17,0.0,1,Girls 13-14 4x100 Meter Relay U15,None,None,Relay,Female,...,4 x 100m relay,44.58,45.4716,46.14030,46.8090,22.17,23.3016,23.97030,24.6390,55.269179
9267,13141,"{995: 'Lee, Asher', 991: 'Yeo, Chee Hean Phili...",29.89,0.0,2,Boys 9-12 4x400 Meter Relay U13,None,None,Relay,Male,...,4 x 400m relay,188.82,192.5964,195.42870,198.2610,29.89,162.7064,165.53870,168.3710,89.170109
9588,13486,"{994: 'De Ming, Gilbert Lee', 988: 'Lee, Jerem...",38.89,0.0,14,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,38.89,1.2572,1.84760,2.4380,6.194106
12424,17876,"{991: 'Yeo, Chee Hean Philip', 992: 'Yeo, Chee...",12.92,0.0,16,Boys 10-12 4x100 Meter Relay U13,None,None,Relay,Male,...,4 x 100m relay,39.36,40.1472,40.73760,41.3280,12.92,27.2272,27.81760,28.4080,72.174797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10975,15886,"., Khairulnazim",14.72,17.0,4,Men 400 Meter Dash Open,2006-01-27,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,14.72,32.8426,33.54205,34.2415,73.432340
9918,13873,"., Kaarthika",12.53,27.0,9,Women 400 Meter Dash Women Junior,1996-06-13,None,Sprint,Female,...,400m,53.84,54.9168,55.72440,56.5320,12.53,42.3868,43.19440,44.0020,81.727340
10701,15018,"., Jayashree",1:14.05,15.0,36,Girls Long Jump B Div,2008-01-29 00:00:00.000,None,Jump,Female,...,Long jump,6.02,5.8996,5.80930,5.7190,10.97,5.0704,5.16070,5.2510,87.225914
12077,17484,"., Cheung Zheng",47.61,14.0,20,Boys 11-14 400 Meter Dash U15,2009-02-19,None,Sprint,Male,...,400m,46.63,47.5626,48.26205,48.9615,47.61,-0.0474,0.65205,1.3515,2.898349


In [447]:
top_performers.to_csv('top_checkpoint.csv', sep=',', encoding='utf-8-sig', index=False)

In [448]:
excluded_list = top_performers.loc[~rslt_df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [449]:
#spexed_list=top_performers

In [450]:
excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
excluded_list['overall_rank'] = 1
excluded_list['overall_rank'] = excluded_list.groupby(['MAPPED_EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list.sort_values(['MAPPED_EVENT', 'GENDER', 'PERF_SCALAR'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_list['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_82118/1842046915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [451]:
#spexed_list=spexed_list[(((spexed_list['EVENT']=='400m')&(spexed_list['overall_rank']<7)))|(((spexed_list['EVENT']=='100m')&(spexed_list['overall_rank']<7)))]

In [452]:
#Apply OCTC selection rule: max 6 for 100m/400m and max 3 for all other events

excluded_list = excluded_list[(((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<7))|(~((excluded_list['MAPPED_EVENT']=='400m')|(excluded_list['MAPPED_EVENT']=='100m'))&(excluded_list['overall_rank']<4))]


In [453]:
excluded_list

,index_x,NAME,RESULT,AGE,COMPETITION_RANK,EVENT_x,DOB,COUNTRY,CATEGORY_EVENT,GENDER,...,Metric,2pc,35pc,5pc,RESULT_CONV,Delta2,Delta35,Delta5,PERF_SCALAR,overall_rank
8135,11374,"Tan, Bernice",2:16.97,23.0,3,Women 10000 Meter Run Open,2000-11-16,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,1
8137,11380,"NicAmhlaoibh, Maire",2:16.97,35.0,2,Women 10000 Meter Run Open,1988-02-14,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,2
8134,11372,"Ng, Yew Cheo",2:16.97,37.0,4,Women 10000 Meter Run Open,1986-11-30,None,Long,Female,...,2131.03,2173.6506,2205.61605,2237.5815,136.97,2036.6806,2068.64605,2100.6115,98.572592,3
9739,13637,"PHAGAMI, KHUM BAHADUR",20.34m,39.0,2,Men 10000 Meter Run Men Senior A,1984-08-09,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,1
9734,13632,"Mohd, Jamian",35.31m,38.0,6,Men 10000 Meter Run Men Senior A,1985-04-29,None,Long,Male,...,1885.55,1923.2610,1951.54425,1979.8275,144.29,1778.9710,1807.25425,1835.5375,97.347591,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3249,3544,SOLANA LEANN REINKIRSTEN CANDA,00:22.7,0.0,9.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,2
3260,3555,SHERYL TOH PEIXUAN,00:23.4,0.0,6.0,Triple Jump,None,None,Jump,Female,...,13.46,13.1908,12.98890,12.7870,1150.60,1137.4092,1137.61110,1137.8130,8453.291233,3
3257,3552,RAO TIANYU,00:28.7,0.0,12.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,1
3246,3541,NG JUN JIE,00:15.7,0.0,11.0,Triple Jump,None,None,Jump,Male,...,15.70,15.3860,15.15050,14.9150,1150.60,1135.2140,1135.44950,1135.6850,7233.662420,2


In [454]:
excluded_list.to_csv('octc_all_events.csv', sep=',', encoding='utf-8-sig', index=False)


In [ ]:
# 1. If top athlete>30 (and the only one >30), there are already 6 in list, and next one is already <30 then it means do nothing? 
# 2. If top athlete>30 and is the sole pick.  Next athlete is far beyond 5% band.  Do we add?
# 3. If top athlete>30 and there are less than 6 within 5% band of SEAG benchmark. Do we add that one more who is beyond 5%?
# 4. At least one per gender.  The only pick is far beyond 5%.  Do we add?

In [ ]:
# Convert time format for marathon and 5000m into mm:ss.00
import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(top_performers_clean)):
        
    rowIndex = top_performers_clean.index[i]

    event=top_performers_clean.iloc[rowIndex,19]
        
    
    time_base2=top_performers_clean.iloc[rowIndex,24]
    time_base3=top_performers_clean.iloc[rowIndex,25]
    time_base5=top_performers_clean.iloc[rowIndex,26]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10000m' or event=='5000m' or event=='3000m steeplechase' or event=='1500m':
        
        print(time_base2, time_base3, time_base5)

        
        
        try:
            
        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
            output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
            output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
       #     print(output2, output3, output5)

                    
       #     top_performers_clean.loc[rowIndex, '2%_timing'] = output2
       #     top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
       #     top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
   
            top_performers_clean.at[rowIndex, '2%'] = output2 # copy over time format
            top_performers_clean.at[rowIndex, '3.5%'] = output3
            top_performers_clean.at[rowIndex, '5%'] = output5


            
        except:
            
            pass
        

        
    elif event=='Marathon':
        
        print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
        #    top_performers_clean.loc[rowIndex, '2%_timing'] = output2
        #    top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
        #    top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            top_performers_clean.at[rowIndex, '2%'] = output2 # copy over time format
            top_performers_clean.at[rowIndex, '3.5%'] = output3
            top_performers_clean.at[rowIndex, '5%'] = output5

            
         #   print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             


# Marinda's request for 6 and 8 place finish analysis

In [4062]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
FROM `saa-analytics.results.saa_full`
WHERE STAGE='Final' AND COMPETITION='SEA Games' AND (RANK='3' OR RANK='6' OR RANK='8')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4064]:
SEAG.tail(60)

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE
53,Farell Glen Felix Jerus,2.15 m,3,High jump,Jump,Male,SEA Games,Final
54,Sun Soklim,NM,6,High jump,Jump,Female,SEA Games,Final
55,Kobsit Sittichai,2.07 m,6,High jump,Jump,Male,SEA Games,Final
56,Suwandi Wijaya,7.47,6,Long jump,Jump,Male,SEA Games,Final
57,Pok Pisey,4.76,8,Long jump,Jump,Female,SEA Games,Final
58,Sapwaturrahman Sapwaturrahman,7.62,3,Long jump,Jump,Male,SEA Games,Final
59,Bùi Thị Loan,6.02,3,Long jump,Jump,Female,SEA Games,Final
60,Nurul Ashikin Abas,5.67,6,Long jump,Jump,Female,SEA Games,Final
61,John Marvin Aragon Rafols,7.19,8,Long jump,Jump,Male,SEA Games,Final
62,Sunisa Khotseemueang,None,3,Heptathlon,Heptathlon,Female,SEA Games,Final


In [247]:
for i in range(len(SEAG)):
        
    rowIndex = SEAG.index[i]

    input_string=SEAG.iloc[rowIndex,3]  # event is column index 3
    
    metric=SEAG.iloc[rowIndex,1] # result is column index 1
    
    if (metric==None or metric=='NH' or metric=='NM'):  # skip those non-numeric values
        continue
        
   # elif metric=='NH':
    #    continue
        
    print(i, input_string, metric)
        
    out = convert_time(i, input_string, metric)
     
    SEAG.loc[rowIndex, 'Metric'] = out

0 1500m     03:59.40
1 1500m     04:04.3
2 1500m     04:05.4
3 Decathlon 6891
4 100m 10.78
5 100m 11.75
6 100m 11.96
7 100m 10.443
8 200m 24.09
9 200m 23.6
11 200m 21.02
12 200m 21.58
13 400m 46.63
14 400m 48.26
15 400m 57.54
16 400m 1:01.20
17 400m 53.84
18 800m 1:57.98
19 800m 1:53.86
20 800m 2:15.77
21 800m 2:18.25
22 800m 2:09.15
23 800m 1:55.39
24 1500m 4:26.33
25 1500m 4:38.64
26 1500m 5:00.090
27 5000m 17:13.63
28 5000m 18:41.76
29 5000m 14:43.45
30 5000m 15:01.79
31 5000m 18:02.52
32 5000m 15:09.65
33 10000m 33:00.23
34 10000m 32:36.32
35 10000m 38:11.04
36 10000m 37:29.58
37 10000m 35:31.03
38 10000m 31:25.55
39 Marathon 2:35:49
40 Marathon 3:17:36
41 Marathon 3:46:44
42 Marathon 2:52:41
43 Marathon 2:50:27
44 Marathon 2:41:36
45 Shot put 11.69
46 Shot put 12.12
47 Shot put 14.44
48 Shot put 14.65
49 Shot put 17.3
50 Shot put 16.06
52 High jump 2.07 m
53 High jump 2.15 m
54 High jump 1.73
55 High jump 1.95 m
56 Long jump 7.47
57 Long jump 5.67
58 Long jump 6.02
59 Long jump 4.

In [248]:
SEAG

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78
...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06


In [249]:
SEAG.to_csv('check_variation.csv', encoding='utf-8')

In [250]:
comps = SEAG[SEAG['RANK']=='3']

In [251]:
comps

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.000
5,Trần Thị Nhi Yến,11.75,3,100m,Sprint,Female,SEA Games,Final,11.750
7,Muhammad Haiqal Hanafi,10.443,3,100m,Sprint,Male,SEA Games,Final,10.443
9,Zaidatul Husniah Zulkifli,23.6,3,200m,Sprint,Female,SEA Games,Final,23.600
11,Lalu Muhammad Zohri,21.02,3,200m,Sprint,Male,SEA Games,Final,21.020
13,Frederick Ramirez,46.63,3,400m,Sprint,Male,SEA Games,Final,46.630
17,Nguyễn Thị Hằng,53.84,3,400m,Sprint,Female,SEA Games,Final,53.840
19,Wan Muhammad Fazri Wan Zahari,1:53.86,3,800m,Mid,Male,SEA Games,Final,113.860
22,Goh Chui Ling,2:09.15,3,800m,Mid,Female,SEA Games,Final,129.150


In [252]:
new_comps = comps[['EVENT', 'GENDER', 'Metric']]

In [253]:
#comps=SEAG.groupby(['EVENT', 'GENDER'])['Metric'].min()


In [254]:
df2 = SEAG.merge(new_comps, on=['EVENT', 'GENDER'], how='left')


In [255]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443
...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050


In [256]:
df2.to_csv('check_metric.csv', encoding='utf-8')

In [257]:
df2['% VARIATION']=(df2['Metric_x']/df2['Metric_y']*100-100)

In [258]:
df2

,NAME,RESULT,RANK,EVENT,CATEGORY_EVENT,GENDER,COMPETITION,STAGE,Metric_x,Metric_y,% VARIATION
0,Wahyudi Putra,03:59.40,3,1500m,Mid,Male,SEA Games,Final,239.40,239.400,0.000000
1,Edwin GIRON,04:04.3,6,1500m,Mid,Male,SEA Games,Final,244.30,239.400,2.046784
2,Van Dung GIANG,04:05.4,8,1500m,Mid,Male,SEA Games,Final,245.40,239.400,2.506266
3,Aries TOLEDO,6891,3,Decathlon,Decathlon,Male,SEA Games,Final,6891.00,6891.000,0.000000
4,Joshua Hanwei Chua,10.78,6,100m,Sprint,Male,SEA Games,Final,10.78,10.443,3.227042
...,...,...,...,...,...,...,...,...,...,...,...
107,"Sukanya Janchaona, Benny Nontanam, Sasipim S...",3:39.29,3,4 x 400m relay,Relay,Female,SEA Games,Final,219.29,219.290,0.000000
108,Nguyễn Thị Huong,11:00.85,3,3000m steeplechase,Steeple,Female,SEA Games,Final,660.85,660.850,0.000000
109,Pandu Sukarya,8:55.05,3,3000m steeplechase,Steeple,Male,SEA Games,Final,535.05,535.050,0.000000
110,Ri Udom,10:36.06,8,3000m steeplechase,Steeple,Male,SEA Games,Final,636.06,535.050,18.878609


In [259]:
df2.to_csv('SEAG_variation.csv', sep=',', encoding='utf-8-sig', index=False)


# Convert timing format from seconds to HMSf

In [4803]:
datetime.datetime.fromtimestamp(120).strftime('%M:%S.%f')
        

'32:00.000000'

In [4808]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [5295]:
import datetime
def format_result(result):
    date = datetime.datetime.utcfromtimestamp(result)
    output = datetime.datetime.strftime(date, "%H:%M:%S.%f")
    return output
print(format_result(113.86))

00:01:53.860000


# Clean manual load data

In [4]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/')


df_SHA = pd.read_csv("SEA_manual_additions.csv", encoding='latin-1')

In [5]:
df_SHA

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,NaN,NaN,NaN,Agus Prayogo,1.0,NaN,NaN,NaN,2:32:59,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
1,NaN,NaN,NaN,ÊArlan Estobo Arbois,2.0,NaN,NaN,NaN,2:33:27,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
2,NaN,NaN,NaN,Nguyen Thanh Hoang,3.0,NaN,NaN,NaN,2:35:49,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
3,NaN,NaN,NaN,Tan Huong Leong,4.0,NaN,NaN,NaN,2:40:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
4,NaN,NaN,NaN,Vanh Pheara,5.0,NaN,NaN,NaN,2:41:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
5,NaN,NaN,NaN,Quoc Luong Trinh,6.0,NaN,NaN,NaN,2:41:36,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
6,NaN,NaN,NaN,Sanchai Namkhet,7.0,NaN,NaN,NaN,2:43:47,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
7,NaN,NaN,NaN,Joanito Fernandes,8.0,NaN,NaN,NaN,2:52:41,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
8,NaN,NaN,NaN,Tony Ah-Thit Payne,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
9,NaN,NaN,NaN,Yang Piseth,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN


In [6]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [7]:
df_SHA.to_csv('SEA_manual_additions.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [2566]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [2567]:
names

,VARIATION,NAME
0,^Harry\sIrfan\sCurra$,HARRY IRFAN CURRAN
1,"Curran, Harry Irfann",HARRY IRFAN CURRAN
2,"^Curran,\sHarry$",HARRY IRFAN CURRAN
3,Harry Irfan Curran Q,HARRY IRFAN CURRAN
4,Wei Jun Huang,HUANG WEI JUN
...,...,...
182,Chua Hsin-Wen Clara,"Chua Hsin-Wen, Clara"
183,Chua Hsin-Wen Clara Q,"Chua Hsin-Wen, Clara"
184,"HSIN-WEN CLARA, Chua","Chua Hsin-Wen, Clara"
185,CHUA HSIN-WEN CLARA,"Chua Hsin-Wen, Clara"


In [ ]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [2568]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    

^Harry\sIrfan\sCurra$
Curran, Harry Irfann
^Curran,\sHarry$
Harry Irfan Curran Q
Wei Jun Huang
HUANG, WEI JUN
., Hariharan
S/O KrishnS/O Krishna, Hariharan
KRISHNAN, HARIHARAN S/O
Andrew George Medina
Medina, Andrew
Medina, Andrew George
George Medina, Andrew
George George Medina, Andrew
Gabriel Lee
Lee, Gabriel
Jun Yu Low
Low Jun Yu
Low, Jun Yu
Caleb Hia
Melvin, Wong
MELVIN, WONG
Conrad Kangli Emery
Conrad, Kangli Emery
Kangli Emery Conrad
Conrad Kangli Emery Q
Mun Jern Wei
Mun, Jern Wei Ivan
Mun, Ivan
Wong Yijie Lucas
Wong, Yijie Lucas
Tung Hon Andrew Pak
Pak, Andrew
Brandon Heng Fu Hong
Heng, Brandon
Brandon Heng Fu Hong
HENG, BRANDON
HENG, FU HONG, BRANDON
HENG, FU HONG BRANDON
Mohamed, Haja Fayiz
Osman, Amir Rusyaidi
nan
Goh, Shaun
Shaun Goh
Ethan Yan
TAN, AARON JUSTIN
Tan, Aaron Justin
Rui Yong Soh
YEE, ERIC CHUN WAI
Yee, Eric
YEE CHUN WAI, ERIC
Eric Yee Chun Wai
Yee Chun Wai, Eric
Yee, Chun Wai, Eric
Yee Chun Wai, Eric
YEE, ERIC CHUN WAI
Lim, Oliver
Jie Cong Jayden, Tan
TAN , JI